### Importação das bibliotecas necessárias para a raspagem de dados.

In [ ]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os
import random
import gzip

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/' # Caminho para a pasta de downloads.
PRE_PROCESSAMENTO = '/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO' # Caminho para a pasta de pré-processamento dos dados.

## Acessando o site Open Street Map e capturando as rotas pendentes e as rotas finalizadas.

In [ ]:
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

conteudo_da_pagina = navegador.page_source
site = BeautifulSoup(conteudo_da_pagina, 'html.parser')
routes = site.findAll('tr')

list_rotas_pendentes = []
list_rotas_finalizadas = []
lista_rotas = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        rotas_pendentes = route.find('span', attrs={'class': 'text-danger'})    
        link_rotas_pendentes = route.find('a')        
        lista_rotas.append([PREFIX_URL_DOWNLOAD+link_rotas_pendentes['href']])
        # list_rotas_pendentes.append([PREFIX_URL_DOWNLOAD+link_rotas_pendentes['href']])
        print (f"{rotas_pendentes.text}: {PREFIX_URL_DOWNLOAD+link_rotas_pendentes['href']}")

    else:         
        link_rotas_finalizadas = route.find('a')        
        lista_rotas.append([PREFIX_URL_DOWNLOAD+link_rotas_finalizadas['href']])
        # list_rotas_finalizadas.append([PREFIX_URL_DOWNLOAD+link_rotas_finalizadas['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+link_rotas_finalizadas['href']}")
time.sleep(3)
navegador.close()

#len(list_rotas_pendentes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_rotas_finalizadas) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Acessando a telas para efetuar os downloads - Renomeando os arquivo adicionando o nome dos usuários - Movendo para a pasta PRE_PROCESSING

In [ ]:
navegador = webdriver.Chrome(service=servico)
users = []
for list_route in lista_rotas:
    time.sleep(3)
    url = list_route[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    user_conteudo_da_pagina = navegador.page_source
    site_user = BeautifulSoup(user_conteudo_da_pagina, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in site_user):
        tb_nome_usuario = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        nome_usuario = tb_nome_usuario.text
        nome_usuario = re.sub(r'\s|\.|\(|\)','_',nome_usuario)        
        print(f'Rota PENDENTE: {list_route} Usuário: {nome_usuario}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(nome_usuario)

    else:
        tb_nome_usuario = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        nome_usuario = tb_nome_usuario.text
        nome_usuario = re.sub(r'\s|\.|\(|\)','_',nome_usuario)
        print(f'Rota FINALIZADA: {list_route} Usuário: {nome_usuario}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(nome_usuario)
time.sleep(3)
navegador.close()

In [ ]:
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gz")]
for file_to_rename in files_to_rename:
    diretorio_origem = os.path.join(DOWNLOADS, file_to_rename)
    diretorio_destino = os.path.dirname(diretorio_origem)
    nome_arquivo_extraido = file_to_rename[:-3]  # Remover a extensão .gz
    with gzip.open(diretorio_origem, 'rb') as arquivo_gz, open(os.path.join(diretorio_destino, nome_arquivo_extraido), 'wb') as arquivo_extraido:
        arquivo_extraido.write(arquivo_gz.read())
    os.remove(diretorio_origem)   
    


    #diretorio_destino = os.path.dirname(caminho_arquivo_gz)
    #with gzip.open(files_to_rename, 'rt') as arquivo:
    #    conteudo = arquivo.read()

In [ ]:
# Corrigindo o nome dos arquivos conforme seus usuários
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        novo_nome = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, novo_nome))

arquivos_para_renomear_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
arquivos_para_renomear_gpx


#for user, arquivo_para_renomear_gpx in zip(users, arquivos_para_renomear_gpx):
users = ['CHEVROLET_BOLT_EUV_2022','CHEVROLET_MALIBU_2022','CHEVROLET_EQUINOX_2022','CHEVROLET_SILVERADO_1500','CHRYSLER_PACIFICA_HYBRID_2019','FORD_MUSTANG','FORD_F-150','FORD_EXPLORER','HONDA_ACCORD_2018','HONDA_ACCORD_HYBRID_2018','HONDA_CIVIC_2016','HONDA_CIVIC_2022','HONDA_PILOT_2017','HYUNDAI_IONIQ_5_2022','HYUNDAI_IONIQ_PHEV_2020','HYUNDAI_KONA_ELECTRIC_2019','HYUNDAI_PALISADE_2020','HYUNDAI_SANTA_FE_2019','HYUNDAI_SANTA_FE_HYBRID_2022','HYUNDAI_SONATA_2020','KIA_EV6_2022','KIA_NIRO_EV_2020','LEXUS_ES_2016','LEXUS_RX_2016','LEXUS_RX_2020','RAM_1500_5TH_GEN','TOYOTA_CAMRY_HYBRID_2021','TOYOTA_COROLLA_HYBRID_TSS2_2019','TOYOTA_COROLLA_TSS2_2019','TOYOTA_HIGHLANDER_2020','TOYOTA_PRIUS_2017','TOYOTA_PRIUS_2017-20','TOYOTA_PRIUS_v_2017','TOYOTA_RAV4_2019','TOYOTA_RAV4_HYBRID_2019','VOLKSWAGEN_GOLF_7TH_GEN','VOLKSWAGEN_PASSAT_2022','VOLKSWAGEN_TIGUAN_2022','VOLKSWAGEN_ATLAS_CROSS_SPORT']
prefix = '__routes'
for arquivo_para_renomear_gpx in arquivos_para_renomear_gpx:
        if os.listdir(DOWNLOADS):
                user = random.choice(users)
                caminho_antigo = os.path.join(DOWNLOADS, arquivo_para_renomear_gpx)        
                #novo_nome = f"{arquivo_para_renomear_gpx.replace('.gpx', '')}__{user}.gpx"
                novo_nome = f"{arquivo_para_renomear_gpx.replace('.gpx', '')}{prefix}__{user}.gpx"                
                caminho_novo = os.path.join(DOWNLOADS,novo_nome)        
                #print(novo_nome)
                os.rename(caminho_antigo, caminho_novo)

In [ ]:
# Movendo os arquivos para a pasta PRE_PROCESSING
#time.sleep(3)
arquivos_para_pre_processamento = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")] # Listando arquivos com extensão [.gpx] (agora renomeados) na pasta de download.
for arquivo_para_pre_processamento in arquivos_para_pre_processamento: # Iterando sobre a lista de arquivos na pasta download que serão movidos para a pasta de pré-processamento.
    caminho_origem = os.path.join(DOWNLOADS, arquivo_para_pre_processamento) # Montando o caminho absoluto da pasta de origem dos arquivos (download + arquivo)
    caminho_destino = os.path.join(PRE_PROCESSAMENTO, arquivo_para_pre_processamento) # Montando o caminho absoluto da pasta de destino dos arquivos (PRE_PROCESSING + arquivo)
    
    try: # Adicionado tratamento de erro para caso não seja possivel a transferencia
        os.rename(caminho_origem,caminho_destino) # Efetivando a transferencia do arquivo da pasta de download para a pasta PRE_PROCESSING
    except Exception as e: #--> Capturando qualquer erro que porventura ocorra
        print(f"Erro ao mover o arquivo: '{arquivo_para_pre_processamento}': {e}.") #--> Exibindo o erro


# 1ª Função. Responsável por fazer o download das rotas diretamente site, renomear os arquivos e os mover para a pasta de pré processamento - Essa execução deve ser executada de uma única vez, pois pode haver alguma falha no momento de renomear os arquivos.

In [ ]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIXO_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/'
PRE_PROCESSAMENTO = '/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO'

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

conteudo_da_pagina = navegador.page_source
site = BeautifulSoup(conteudo_da_pagina, 'html.parser')
rotas = site.findAll('tr')

lista_rotas_pendentes = []
lista_rotas_finalizadas = []
lista_rotas = []

for rota in rotas:
    if rota.find('span', attrs={'class': 'text-danger'}):
        rotas_pendentes = rota.find('span', attrs={'class': 'text-danger'})    
        link_rotas_pendentes = rota.find('a')        
        lista_rotas.append([PREFIXO_URL_DOWNLOAD+link_rotas_pendentes['href']])
        # lista_rotas_pendentes.append([PREFIX_URL_DOWNLOAD+link_rotas_pendentes['href']])
        print (f"{rotas_pendentes.text}: {PREFIXO_URL_DOWNLOAD+link_rotas_pendentes['href']}")

    else:         
        link_rotas_finalizadas = rota.find('a')        
        lista_rotas.append([PREFIXO_URL_DOWNLOAD+link_rotas_finalizadas['href']])
        # lista_rotas_finalizadas.append([PREFIX_URL_DOWNLOAD+link_rotas_finalizadas['href']])
        print(f"FINISHED: {PREFIXO_URL_DOWNLOAD+link_rotas_finalizadas['href']}")
time.sleep(3)
navegador.close()

#len(list_rotas_pendentes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_rotas_finalizadas) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
#navegador = webdriver.Chrome(service=servico)


## Baixando os arquivos 
navegador = webdriver.Chrome(service=servico)
usuarios = []
for lista_rota in lista_rotas:
    time.sleep(3)
    url = lista_rota[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    conteudo_pagina_download = navegador.page_source
    pagina_usuario = BeautifulSoup(conteudo_pagina_download, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in pagina_usuario):
        tb_nome_usuario = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        nome_usuario = tb_nome_usuario.text
        nome_usuario = re.sub(r'\s|\.|\(|\)','_',nome_usuario)        
        print(f'Rota PENDENTE: {lista_rota} Usuário: {nome_usuario}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        usuarios.append(nome_usuario)

    else:
        tb_nome_usuario = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        nome_usuario = tb_nome_usuario.text
        nome_usuario = re.sub(r'\s|\.|\(|\)','_',nome_usuario)
        print(f'Rota FINALIZADA: {lista_rota} Usuário: {nome_usuario}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        usuarios.append(nome_usuario)
time.sleep(5)
navegador.close()


## Corrigindo o nome dos arquivos conforme seus usuários
arquivos_para_renomear = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for arquivo_para_renomear in arquivos_para_renomear:
        novo_nome = arquivo_para_renomear.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, arquivo_para_renomear), os.path.join(DOWNLOADS, novo_nome))

arquivos_para_renomear_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
arquivos_para_renomear_gpx

users = ['CHEVROLET_BOLT_EUV_2022','CHEVROLET_MALIBU_2022','CHEVROLET_EQUINOX_2022','CHEVROLET_SILVERADO_1500','CHRYSLER_PACIFICA_HYBRID_2019','FORD_MUSTANG','FORD_F-150','FORD_EXPLORER','HONDA_ACCORD_2018','HONDA_ACCORD_HYBRID_2018','HONDA_CIVIC_2016','HONDA_CIVIC_2022','HONDA_PILOT_2017','HYUNDAI_IONIQ_5_2022','HYUNDAI_IONIQ_PHEV_2020','HYUNDAI_KONA_ELECTRIC_2019','HYUNDAI_PALISADE_2020','HYUNDAI_SANTA_FE_2019','HYUNDAI_SANTA_FE_HYBRID_2022','HYUNDAI_SONATA_2020','KIA_EV6_2022','KIA_NIRO_EV_2020','LEXUS_ES_2016','LEXUS_RX_2016','LEXUS_RX_2020','RAM_1500_5TH_GEN','TOYOTA_CAMRY_HYBRID_2021','TOYOTA_COROLLA_HYBRID_TSS2_2019','TOYOTA_COROLLA_TSS2_2019','TOYOTA_HIGHLANDER_2020','TOYOTA_PRIUS_2017','TOYOTA_PRIUS_2017-20','TOYOTA_PRIUS_v_2017','TOYOTA_RAV4_2019','TOYOTA_RAV4_HYBRID_2019','VOLKSWAGEN_GOLF_7TH_GEN','VOLKSWAGEN_PASSAT_2022','VOLKSWAGEN_TIGUAN_2022','VOLKSWAGEN_ATLAS_CROSS_SPORT']
prefix = '__routes'

# for usuario, arquivo_para_renomear_gpx in zip(usuarios, arquivos_para_renomear_gpx):        
#     caminho_antigo = os.path.join(DOWNLOADS, arquivo_para_renomear_gpx)
#     novo_nome = f"{arquivo_para_renomear_gpx.replace('.gpx', '')}__{usuario}.gpx"        
#     caminho_novo = os.path.join(DOWNLOADS,novo_nome)            
#     os.rename(caminho_antigo, caminho_novo)

for arquivo_para_renomear_gpx in arquivos_para_renomear_gpx:    
    user = random.choice(users)
    caminho_antigo = os.path.join(DOWNLOADS, arquivo_para_renomear_gpx)
    novo_nome = f"{arquivo_para_renomear_gpx.replace('.gpx', '')}{prefix}__{user}.gpx"                
    caminho_novo = os.path.join(DOWNLOADS,novo_nome) # Montando o novo caminho absoluto da pasta download + arquivo renomeado
    os.rename(caminho_antigo, caminho_novo) # Efetivando a renomeação do arquivo antigo pelo novo


# Movendo os arquivos para a pasta PRE_PROCESSING
time.sleep(3)
arquivos_para_pre_processamento = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")]
for arquivo_para_pre_processamento in arquivos_para_pre_processamento:
    caminho_origem = os.path.join(DOWNLOADS, arquivo_para_pre_processamento)
    caminho_destino = os.path.join(PRE_PROCESSAMENTO, arquivo_para_pre_processamento)
    try:
        os.rename(caminho_origem,caminho_destino)
        print(f"Arquivo '{arquivo_para_pre_processamento}' movido para a pasta {PRE_PROCESSAMENTO}.")
    except Exception as e:
        print(f"Erro ao mover o arquivo: '{arquivo_para_pre_processamento}': {e}.")

In [ ]:
import time
start_time = time.time()

In [ ]:
end_time = time.time()

In [ ]:
execution_time = end_time - start_time

In [ ]:
hours, remainder = divmod(execution_time, 3600)
minutes, remainder = divmod(remainder, 60)
seconds, milliseconds = divmod(remainder, 1)

In [ ]:
print(f"Tempo de execução: {int(hours)} horas, {int(minutes)} minutos, {int(seconds)} segundos e {int(milliseconds * 1000)} milissegundos")


# 2ª Função - Lendo arquivos pasta de Pré Processamento - Convertendo os arquivos .gpx em .csv - Levando os arquivos .csv para a camada bronze do datalake

In [ ]:
##############################################
### IMPORTAÇÃO DAS BIBLIOTECAS NECESSÁRIAS ###
##############################################
# pip install minio 
# pip install gpxpy
# pip install pandas 

import gpxpy 
import gpxpy.gpx 
import pandas as pd 
import os 
from minio import Minio
from minio.error import S3Error

##############################
### DEFINIÇÃO DE VARIÁVEIS ### 
##############################
PRE_PROCESSAMENTO = '/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO' 
CAMADA_BRONZE = 'bronze' 

##############################################
### CRIANDO UMA INSTÂNCIA DO CLIENTE MINIO ###
##############################################
minioclient = Minio('localhost:9000', 
    access_key='minioadmin', 
    secret_key='minioadmin', 
    secure=False) 

###########################################################
### CONVERTENDO OS ARQUIVOS COM A EXTENSÃO .GPX EM .CSV ###
###########################################################
arquivos_pre_processamento = [arquivo for arquivo in os.listdir(PRE_PROCESSAMENTO) if arquivo.endswith(".gpx")] 
for arquivo_pre_processamento in arquivos_pre_processamento: 
    caminho_arquivo = os.path.join(PRE_PROCESSAMENTO, arquivo_pre_processamento) 
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo_gpx: 
            gpx = gpxpy.parse(arquivo_gpx)

        info_rota = [] 
        for trilha in gpx.tracks: 
            for segmento in trilha.segments:
                for ponto in segmento.points:
                    info_rota.append({
                        'latitude': ponto.latitude, 
                        'longitude': ponto.longitude, 
                        'elevacao' : ponto.elevation, 
                        'time_point' : ponto.time 
                    })
        arquivo_pre_processamento_csv = arquivo_pre_processamento.replace('.gpx','.csv') 
        info_rota_df = pd.DataFrame(info_rota) 
        info_rota_df.to_csv(f'{PRE_PROCESSAMENTO}/{arquivo_pre_processamento_csv}', index=False) 
        os.remove(caminho_arquivo) 
    except Exception as e: 
        print(f"Erro ao converter o '{arquivo_pre_processamento}': {e}.") 
        os.remove(caminho_arquivo)
        print(f"Excluindo o arquivo defeituoso: {arquivo_pre_processamento}")
        

################################################################################################################
### MOVENDO OS ARQUIVOS COM A EXTENSÃO .CSV DA PASTA PRE_PROCESSAMENTO PARA A PRIMEIRA CAMADA (BRONZE) NO MINIO ###
################################################################################################################
arquivos_para_datalake = [arquivo for arquivo in os.listdir(PRE_PROCESSAMENTO) if arquivo.endswith(".csv")] 
for nome_arquivo in arquivos_para_datalake: 
    caminho_pre_proc = os.path.join(PRE_PROCESSAMENTO, nome_arquivo) 
    if os.path.isfile(caminho_pre_proc): 
        try:
            minioclient.fput_object(CAMADA_BRONZE, nome_arquivo, caminho_pre_proc) 
            print(f"Arquivo {nome_arquivo} enviado com sucesso para o bucket.") 
            os.remove(caminho_pre_proc) 
        except S3Error as e: 
            print(f"Erro ao enviar o arquivo: {nome_arquivo} -> Erro: {e}") 

In [ ]:
with open('/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO/10086735__Routes_from_dragonpilot_2023_08_15__TOYOTA_RAV4_HYBRID_2019__.gpx', 'r', encoding='utf-8') as arquivo_gpx:
    gpx = gpxpy.parse(arquivo_gpx)

geolocator = Nominatim(user_agent="geoapiExercises")
info_rota = [] #--> Lista que servirá de apoio para converter o arquivo .gpx em uma lista
for trilha in gpx.tracks:
    for segmento in trilha.segments:
        for ponto in segmento.points:            
            
            latitude = ponto.latitude
            longitude = ponto.longitude

            address = location.raw['address']
            city = address.get('city', '')
            state = address.get('state', '')
            country = address.get('country', '')

            location = geolocator.reverse(f'{latitude},{longitude}')
            address = location.raw['address']
            cidade = address.get('suburb')
            estado = address.get('state')
            pais = address.get('country_code')

            info_rota.append({
                'latitude': ponto.latitude,
                'longitude': ponto.longitude,
                'elevacao' : ponto.elevation,
                'time_point' : ponto.time,
                'cidade': cidade,
                'estado': estado,
                'pais': pais
            })
arquivo_pre_processamento_csv = arquivo_pre_processamento.replace('.gpx','.csv')
info_rota_df = pd.DataFrame(info_rota)
## os.remove(caminho_arquivo)

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.reverse(f'{Latitude},{Longitude}') 

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
Latitude = "45.24559097710776"
Longitude = "-122.79781984274484"
location = geolocator.reverse(f'{Latitude},{Longitude}') 


address = location.raw['address']
cidade = address.get('suburb')
estado = address.get('ISO3166-2-lvl4')
pais = address.get('country_code')

estado = estado[-2:]
pais = pais.upper()

print(f"Cidade: {cidade}")
print(f"Estado (abreviado): {estado[-2:]}")
print(f"País: {pais}")

In [ ]:
df.head()

In [ ]:
df.to_csv('arquivo.csv', index=False, sep=';')

In [ ]:
PADRAO_REGEX = r'(.*?)__'
padrao_encontrado = re.findall(PADRAO_REGEX, nome_arquivo)
id_rota = padrao_encontrado[0]
nome_usuario = padrao_encontrado[-1]
padrao_encontrado

In [ ]:
arquivos_para_datalake = [files for files in os.listdir(PRE_PROCESSAMENTO) if files.endswith(".csv")]
#arquivos_para_datalake = [files for files in os.listdir(PRE_PROCESSAMENTO) if files.endswith(".parquet")]
len(arquivos_para_datalake)

In [ ]:
## Aqui os arquivos com a extensão .csv saem da pasta PRE_PROCESSING para a primeira camada (bronze) no MinIO
#arquivos_para_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".csv")]
arquivos_para_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".parquet")]
for nome_arquivo in arquivos_para_datalake:
    caminho_pre_proc = os.path.join(PRE_PROCESSING, nome_arquivo)    
    if os.path.isfile(caminho_pre_proc):
        try:
            minioclient.fput_object(BRONZE_LAYER, nome_arquivo, caminho_pre_proc)
            print(f"Arquivo {nome_arquivo} enviado com sucesso para o bucket.")
            os.remove(caminho_pre_proc) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta download
        except S3Error as e:
            print(f"Erro ao enviar o arquivo: {nome_arquivo} -> Erro: {e}")     

In [ ]:
with open('PRE_PROCESSING/9632214__Routes_from_sunnypilot_2023_08_19-dev__SUBARU_IMPREZA_LIMITED_2019__.gpx', 'r', encoding='utf-8') as arquivo_gpx:
    gpx = gpxpy.parse(arquivo_gpx)

In [ ]:
arquivo_gpx = '9632229__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx'

In [ ]:
arquivo_gpx = str(arquivo_gpx)

In [ ]:
type(arquivo_gpx)

In [ ]:
with open(arquivo_gpx, 'r', encoding='utf-8') as arquivo_gpx:
    gpx = gpxpy.parse(arquivo_gpx)

In [ ]:
gpx

In [ ]:
type(gpx)

In [ ]:
# Para retornar a quantidade de pontos (lat+lon+ele) em um arquivo arquivo '.gpx' podemos usar get
# Documentação: https://pypi.org/project/gpxpy/
gpx.get_track_points_no()

In [ ]:
# Para retornar a faixa de altitude, a fim de se obter extremos de maior elevação e menor elevação no trajeto percorrido - os valores apresentados são em metros acima do nível do mar
gpx.get_elevation_extremes()

In [ ]:
# Não endendi para que isso serve
gpx.get_uphill_downhill()

In [ ]:
# Obtendo o nome do criador da rota - Entretanto no nosso caso quando usamos esse comando temos uma informação generica - Foi necessário obter o nome do criador da rota com o WebScraping
creator = gpx.creator
creator

In [ ]:
# Para exibir o conteudo do arquivo .gpx em formato xml
print(gpx.to_xml()[:1000])

In [ ]:
# É possivel verificar quantas rotas/trilhas nosso arquivo .gpx possui
len(gpx.tracks)

In [ ]:
# Como no nosso exemplo acima só temos 1 rota/trilha efetuada, podemos acessar por meio de indice no python, 
# Nesse caso se passa o valor [0] para ter uma precisão exata da rota em que estamos trabalhando
gpx.tracks[0]

In [ ]:
# Agora vamos acessar os segmentos da nossa rota/trilha
gpx.tracks[0].segments[0]

In [ ]:
# Agora podemos acessar os pontos de dados individuais dentro do nosso gpx acessando a matriz de pontos. Aqui tambem podemo0s ver o nome das propriedades do arquivo como elevation e time
gpx.tracks[0].segments[0].points[:10]

In [ ]:
# Aqui acessamos as tres camadas: Track - Segments e Point e o passamos para um array em forma de dicionário. Note que podemos acessar cada atributo. No meu caso estou pegando latitude, Longitude, Elevação e a hora de cada ponto (entenda-se como ponto Lat+Long+Ele)
info_rota = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            info_rota.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevacao' : point.elevation,
                'time_point' : point.time
            })

In [ ]:
# Exibindo somente os 3 primeiros resultados
info_rota[:3]

In [ ]:
type(info_rota)

In [ ]:
# Aqui podemos transformar essa informação em um dataframe com a biblioteca pandas
info_rota_df = pd.DataFrame(info_rota)
info_rota_df.head()

In [ ]:
type(info_rota_df)

In [ ]:
# Aqui salvamos o pandas dataframe localmente
info_rota_df.to_csv('pandas_info_rota.csv', index=False)

# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

In [ ]:
# Aqui salvamos o pandas dataframe localmente
info_rota_df.to_parquet('pd_parquet_info_rota.parquet', index=False)

# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

# 3ª Função - Lendo o bucket e transformando os arquivos encontrados lá em DF Pandas

In [6]:
# pip install minio
# pip install pandas
# pip install numpy

from minio import Minio
from minio.error import S3Error
from io import StringIO, BytesIO
import pandas as pd
import re
import numpy as np
from scipy.spatial.distance import euclidean, cityblock, minkowski





CAMADA_BRONZE = 'bronze'
CAMADA_SILVER = 'silver'
PADRAO_REGEX = r'(.*?)__'

PADRAO_REGEX_1 = r'(\d+)__(.*?)\.csv'
PADRAO_REGEX_2 = r'(.*?)__'
PADRAO_REGEX_3 = '__.+'



minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_BRONZE) if arquivo_gpx.object_name.endswith(".csv")]
arquivos_rotas_gpx_csv

In [ ]:
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:
    try:

        ### OBTENDO O ARQUIVO E O CONVERTENDO EM UM DF PANDAS ###
        obj_rota_csv = minioclient.get_object(CAMADA_BRONZE, arquivo_rotas_gpx_csv.object_name)        
        csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string    
        arquivo_csv = StringIO(csv_decod)
        df = pd.read_csv(arquivo_csv, sep=';')

        #### SEPARANDO A INFORMAÇÃO DE DATA E HORA EM 2 COLUNAS SEPARADAS ###
        df['time_point'] = pd.to_datetime(df['time_point'], format='%Y-%m-%d %H:%M:%S', errors='coerce')        

        df['data'] = df['time_point'].dt.date
        df['hora'] = df['time_point'].dt.strftime('%H:%M:%S')
               
        
        df = df.drop(columns=['time_point'])
    except S3Error as e:
        print(f"Erro ao alterar a data do arquivo: {nome_arquivo}. Erro: {e}")

In [ ]:
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:


    ### OBTENDO O ARQUIVO E O CONVERTENDO EM UM DF PANDAS ###
    obj_rota_csv = minioclient.get_object(CAMADA_BRONZE, arquivo_rotas_gpx_csv.object_name)        
    csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string    
    arquivo_csv = StringIO(csv_decod)
    df = pd.read_csv(arquivo_csv, sep=';')

    #### SEPARANDO A INFORMAÇÃO DE DATA E HORA EM 2 COLUNAS SEPARADAS ###
    df['time_point'] = pd.to_datetime(df['time_point'], errors='coerce')
    df['data'] = df['time_point'].dt.date
    df['hora'] = df['time_point'].dt.strftime('%H:%M:%S')
    df = df.drop(columns=['time_point'])

df.head()


In [ ]:
pd.to_datetime(df['time_point'])      

In [ ]:
df

In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_BRONZE) if arquivo_gpx.object_name.endswith(".csv")]
arquivos_rotas_gpx_csv

In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_BRONZE) if arquivo_gpx.object_name.endswith(".csv")]
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:

    ### OBTENDO O ARQUIVO E O CONVERTENDO EM UM DF PANDAS ###
    obj_rota_csv = minioclient.get_object(CAMADA_BRONZE, arquivo_rotas_gpx_csv.object_name)        
    csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string    
    arquivo_csv = StringIO(csv_decod)
    df = pd.read_csv(arquivo_csv, sep=';')

    ### SEPARANDO A INFORMAÇÃO DE DATA E HORA EM 2 COLUNAS SEPARADAS ###
    # df['time_point'] = pd.to_datetime(df['time_point'])
    # df['data'] = df['time_point'].dt.date
    # df['hora'] = df['time_point'].dt.strftime('%H:%M:%S')
    # df = df.drop(columns=['time_point']) 

    ### USANDO O REGEX PARA PEGAR O ID DA RODA E O NOME DO USUÁRIO (DADOS PRESENTES NO NOME DO ARQUIVO) ###
    nome_arquivo = arquivo_rotas_gpx_csv.object_name        
    padrao_encontrado = re.findall(PADRAO_REGEX_3, nome_arquivo)
    id_rota = padrao_encontrado[0][0]
    nome_usuario = padrao_encontrado[0][1]
    num_df = len(df)
    replic_id_rota = np.tile(id_rota, num_df)
    replic_nome_usuario = np.tile(nome_usuario, num_df)
    # df['id_rota'] = replic_id_rota
    # df['nome_usuario'] = replic_nome_usuario
    # ordenacao_df = ['id_rota', 'nome_usuario', 'latitude', 'longitude', 'elevacao', 'data', 'hora']
    # df = df.reindex(columns=ordenacao_df)

    #try:
    #    csv_bytes = df.to_csv(index=False).encode('utf-8') 
    #    csv_buffer = BytesIO(csv_bytes)
    #    minioclient.put_object(
    #                        CAMADA_SILVER,
    #                        nome_arquivo,
    #                        data=csv_buffer,
    #                        length=len(csv_bytes),
    #                        content_type='application/csv')

        #minioclient.remove_object(CAMADA_BRONZE, nome_arquivo)        

In [ ]:
id_rota

In [ ]:
PADRAO_REGEX_1 = r'(\d+)__(.*?)\.csv'
PADRAO_REGEX_2 = r'(.*?)__'
PADRAO_REGEX_3 = '__.+'
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_BRONZE) if arquivo_gpx.object_name.endswith(".csv")]
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:
    nome_arquivo = arquivo_rotas_gpx_csv.object_name
    padrao_encontrado = re.search(PADRAO_REGEX_1, nome_arquivo)  
    if padrao_encontrado:
        id_rota = padrao_encontrado.group(1)        
        nome_usuario = padrao_encontrado.group(2)        
        substrings = re.findall(PADRAO_REGEX_3, nome_usuario)                
        nome_usuario = [substring.replace('__','') for substring in substrings]
        print(nome_usuario)

#        if substrings:
#            nome_usuario = substrings[-1]
#        else:
#            nome_usuario = nome_usuario       
#        print(f"ID da Rota: {id_rota}")
#        print(f"Nome do Usuário: {nome_usuario}")


In [ ]:
padrao_encontrado

In [ ]:
blob_rota_csv = minioclient.get_object(BRONZE_LAYER, '9781413__Routes_from_dragonpilot_2023_07_05__TOYOTA_COROLLA_TSS2_2019__.csv')
rota_csv = blob_rota_csv.data
rota_csv = rota_csv.decode("ISO-8859-1")
pre_df = StringIO(rota_csv)
df = pd.read_csv(pre_df)

In [ ]:
df.head()

In [ ]:
df['time_point'] = pd.to_datetime(df['time_point'])
df['data'] = df['time_point'].dt.date
df['hora'] = df['time_point'].dt.strftime('%H:%M:%S')
df = df.drop(columns=['time_point'])

In [ ]:
df.head()

In [ ]:
import re
import numpy as np


text = "9781892__Routes_from_sunnypilot_0_9_3_1-release__HONDA_ACCORD_2018__.gpx"

# Definindo o padrão de regex
pattern = r'(.*?)__'

# Encontrando todas as correspondências no texto
matches = re.findall(pattern, text)

# Extraindo o nome do carro
username = matches[-1]
id = matches[0]

print(f'Id: {id} - username: {username}')


In [ ]:
num_df = len(df)
num_df

In [ ]:
repeated_id = np.tile(id, num_df)
repeated_user = np.tile(username, num_df)


In [ ]:
df['id'] = repeated_id
df['username'] = repeated_user


In [ ]:
# Reordenando as colunas
desired_order = ['id', 'username', 'latitude', 'longitude', 'elevacao', 'data', 'hora']
df = df.reindex(columns=desired_order)

In [ ]:
df.head()

In [ ]:
df.to_csv('arquivo.csv', index=False, sep=';')

In [ ]:
import re

# Suas strings de exemplo
strings = [
    "10109849__Muara_Bulian.csv",
    "10109852__Routes_from_sunnypilot_0_9_4_1-release__VOLKSWAGEN_PASSAT_8TH_GEN__.csv",
    "10127398__Jambi_Paradise_-_Simpang_Paal_Merah.csv"
]

# Padrão regex
PADRAO_REGEX_1 = r'(\d+)__(.*?)\.csv'
PADRAO_REGEX_2 = r'(.*?)__'


In [ ]:
for nome_arquivo in strings:
    padrao_encontrado = re.search(PADRAO_REGEX_1, nome_arquivo)
    if padrao_encontrado:
        id_rota = padrao_encontrado.group(1)
        nome_usuario = padrao_encontrado.group(2)

        substrings = re.findall(PADRAO_REGEX_2, nome_usuario)
        if substrings:
            nome_usuario = substrings[-1]
        else:
            nome_usuario = nome_usuario
       
        print(f"ID da Rota: {id_rota}")
        print(f"Nome do Usuário: {nome_usuario}")

In [ ]:
import re

# Suas strings de exemplo
strings = [
    "10109849__Muara_Bulian.csv",
    "10109852__Routes_from_sunnypilot_0_9_4_1-release__VOLKSWAGEN_PASSAT_8TH_GEN__.csv",
    "10127398__Jambi_Paradise_-_Simpang_Paal_Merah.csv"
]

# Padrão regex
PADRAO_REGEX_1 = r'(\d+)__(.*?)\.csv'
PADRAO_REGEX_2 = r'(.*?)__'

for nome_arquivo in strings:
    padrao_encontrado = re.search(PADRAO_REGEX_1, nome_arquivo)
    if padrao_encontrado:
        id_rota = padrao_encontrado.group(1)
        nome_usuario = padrao_encontrado.group(2)

        substrings = re.findall(PADRAO_REGEX_2, nome_usuario)
        if substrings:
            nome_usuario = substrings[-1]
        else:
            nome_usuario = nome_usuario
       
        print(f"ID da Rota: {id_rota}")
        print(f"Nome do Usuário: {nome_usuario}")


In [51]:
from scipy.spatial.distance import euclidean, cityblock, minkowski

df = pd.read_csv('11309115__routes__HONDA_CIVIC_2022.csv', sep=';')

df['lat1'] = df['latitude'].shift() #--> Aqui eu passo a função shift na coluna latitude e já crio uma nova coluna chamada lat 1

df['lon1'] = df['longitude'].shift() #--> Aqui eu passo a função shift na coluna longitude e já crio uma nova coluna chamada lon 1

df = df.rename(columns={'latitude': 'lat2', 'longitude': 'lon2'})

df

# Observe que a primeira linha fica com valores vazios/nan pq não existe nenhum valor acima do 38.473332	144.927545
# essa função .shift() pega o resultado de uma coluna de uma linha e joga pra uma uma linha abaixo. Nesse exemplo eu criei 2 colunas novas

,id_rota,nome_usuario,lat2,lon2,elevacao,data,hora,cidade,estado,pais,data_carga_banco,lat1,lon1
0,11309115,HONDA_CIVIC_2022,25.405903,-100.134304,456.357153,2024-03-18,16:17:31,NaN,MX-NLE,MX,2024-03-19 21:54:02,NaN,NaN
1,11309115,HONDA_CIVIC_2022,25.405721,-100.134209,456.267968,2024-03-18,16:17:32,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.405903,-100.134304
2,11309115,HONDA_CIVIC_2022,25.405539,-100.134116,456.450866,2024-03-18,16:17:33,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.405721,-100.134209
3,11309115,HONDA_CIVIC_2022,25.405356,-100.134023,456.304257,2024-03-18,16:17:34,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.405539,-100.134116
4,11309115,HONDA_CIVIC_2022,25.405174,-100.133930,456.173852,2024-03-18,16:17:35,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.405356,-100.134023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7014,11309115,HONDA_CIVIC_2022,25.658215,-100.372543,603.861116,2024-03-18,21:06:54,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.658173,-100.372397
7015,11309115,HONDA_CIVIC_2022,25.658258,-100.372691,603.889631,2024-03-18,21:06:55,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.658215,-100.372543
7016,11309115,HONDA_CIVIC_2022,25.658302,-100.372841,604.039239,2024-03-18,21:06:56,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.658258,-100.372691
7017,11309115,HONDA_CIVIC_2022,25.658348,-100.372990,604.363518,2024-03-18,21:06:57,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.658302,-100.372841


In [52]:
df = df.dropna(subset=['lat1', 'lon1'])
df
# Aqui eu dropo os valores nulos, ou seja puxo o resultado da linha 2 pra linha 1 nas colunas lat1 e lon1

,id_rota,nome_usuario,lat2,lon2,elevacao,data,hora,cidade,estado,pais,data_carga_banco,lat1,lon1
1,11309115,HONDA_CIVIC_2022,25.405721,-100.134209,456.267968,2024-03-18,16:17:32,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.405903,-100.134304
2,11309115,HONDA_CIVIC_2022,25.405539,-100.134116,456.450866,2024-03-18,16:17:33,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.405721,-100.134209
3,11309115,HONDA_CIVIC_2022,25.405356,-100.134023,456.304257,2024-03-18,16:17:34,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.405539,-100.134116
4,11309115,HONDA_CIVIC_2022,25.405174,-100.133930,456.173852,2024-03-18,16:17:35,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.405356,-100.134023
5,11309115,HONDA_CIVIC_2022,25.404990,-100.133838,455.926006,2024-03-18,16:17:36,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.405174,-100.133930
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7014,11309115,HONDA_CIVIC_2022,25.658215,-100.372543,603.861116,2024-03-18,21:06:54,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.658173,-100.372397
7015,11309115,HONDA_CIVIC_2022,25.658258,-100.372691,603.889631,2024-03-18,21:06:55,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.658215,-100.372543
7016,11309115,HONDA_CIVIC_2022,25.658302,-100.372841,604.039239,2024-03-18,21:06:56,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.658258,-100.372691
7017,11309115,HONDA_CIVIC_2022,25.658348,-100.372990,604.363518,2024-03-18,21:06:57,NaN,MX-NLE,MX,2024-03-19 21:54:02,25.658302,-100.372841


In [10]:
def calc_dist_euclidiana(row):
    coord1 = (float(row['lon1']), float(row['lat1']))
    coord2 = (float(row['lon2']), float(row['lat2']))
    return euclidean(coord1, coord2)*1000

def calc_dist_manhattan(row):
    coord1 = (float(row['lon1']), float(row['lat1']))
    coord2 = (float(row['lon2']), float(row['lat2']))
    return cityblock(coord1, coord2)*1000

def calc_dist_minkowski(row, p=3):
    coord1 = (float(row['lon1']), float(row['lat1']))
    coord2 = (float(row['lon2']), float(row['lat2']))
    return minkowski(coord1, coord2, p)*1000

# def haversine_distance(row):
#     lat1 = float(row['lat1'])
#     lon1 = float(row['lon1'])
#     lat2 = float(row['lat2'])
#     lon2 = float(row['lon2'])
    
#     if not (-90 <= lat1 <= 90) or not (-90 <= lat2 <= 90):
#         raise ValueError("Latitude must be in the [-90; 90] range.")
    
#     coord1 = (lon1, lat1)
#     coord2 = (lon2, lat2)
#     return geodesic(coord1, coord2).kilometers



# def haversine(lat1, lon1, lat2, lon2):
#     R = 6371  # Raio da Terra em km
#     d_lat = radians(lat2 - lat1)
#     d_lon = radians(lon2 - lon1)
#     a = sin(d_lat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(d_lon / 2) ** 2
#     c = 2 * atan2(sqrt(a), sqrt(1 - a))
#     distance = R * c
#     return distance

In [11]:
df['dist_euclidiana'] = df.apply(calc_dist_euclidiana, axis=1)
df['dist_manhattan'] = df.apply(calc_dist_manhattan, axis=1)
df['dist_minkowski'] = df.apply(calc_dist_minkowski, axis=1)
#df['dist_haversine'] = df.apply(lambda row: haversine(row['lat1'], row['lon1'], row['lat2'], row['lon2']), axis=1)

/tmp/ipykernel_4140/18453585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dist_euclidiana'] = df.apply(calc_dist_euclidiana, axis=1)
/tmp/ipykernel_4140/18453585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dist_manhattan'] = df.apply(calc_dist_manhattan, axis=1)
/tmp/ipykernel_4140/18453585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [12]:
df

,id_rota,nome_usuario,lat2,lon2,elevacao,data,hora,cidade,estado,pais,data_carga_banco,lat1,lon1,dist_euclidiana,dist_manhattan,dist_minkowski
1,11190523,HYUNDAI_IONIQ_5_2022,-38.473334,144.927550,93.2,2023-11-21,03:58:26,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473332,144.927545,0.005385,0.007,0.005104
2,11190523,HYUNDAI_IONIQ_5_2022,-38.473336,144.927554,93.2,2023-11-21,03:58:27,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473334,144.927550,0.004472,0.006,0.004160
3,11190523,HYUNDAI_IONIQ_5_2022,-38.473333,144.927554,93.2,2023-11-21,03:58:28,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473336,144.927554,0.003000,0.003,0.003000
4,11190523,HYUNDAI_IONIQ_5_2022,-38.473344,144.927568,93.2,2023-11-21,03:58:30,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473333,144.927554,0.017804,0.025,0.015973
5,11190523,HYUNDAI_IONIQ_5_2022,-38.473353,144.927583,93.3,2023-11-21,03:58:31,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473344,144.927568,0.017493,0.024,0.016010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7647,11190523,HYUNDAI_IONIQ_5_2022,-38.477056,144.893626,126.8,2023-11-21,06:25:50,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.477057,144.893619,0.007071,0.008,0.007007
7648,11190523,HYUNDAI_IONIQ_5_2022,-38.477054,144.893628,126.8,2023-11-21,06:25:51,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.477056,144.893626,0.002828,0.004,0.002520
7649,11190523,HYUNDAI_IONIQ_5_2022,-38.477053,144.893628,126.8,2023-11-21,06:25:52,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.477054,144.893628,0.001000,0.001,0.001000
7650,11190523,HYUNDAI_IONIQ_5_2022,-38.477053,144.893627,126.8,2023-11-21,06:25:53,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.477053,144.893628,0.001000,0.001,0.001000


In [6]:
df.head()

,id_rota,nome_usuario,lat2,lon2,elevacao,data,hora,cidade,estado,pais,data_carga_banco,lat1,lon1,dist_euclidiana,dist_manhattan,dist_minkowski
1,11190523,HYUNDAI_IONIQ_5_2022,-38.473334,144.927550,93.2,2023-11-21,03:58:26,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473332,144.927545,0.005385,0.007,0.005104
2,11190523,HYUNDAI_IONIQ_5_2022,-38.473336,144.927554,93.2,2023-11-21,03:58:27,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473334,144.927550,0.004472,0.006,0.004160
3,11190523,HYUNDAI_IONIQ_5_2022,-38.473333,144.927554,93.2,2023-11-21,03:58:28,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473336,144.927554,0.003000,0.003,0.003000
4,11190523,HYUNDAI_IONIQ_5_2022,-38.473344,144.927568,93.2,2023-11-21,03:58:30,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473333,144.927554,0.017804,0.025,0.015973
5,11190523,HYUNDAI_IONIQ_5_2022,-38.473353,144.927583,93.3,2023-11-21,03:58:31,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473344,144.927568,0.017493,0.024,0.016010


In [12]:
import pandas as pd
import numpy as np
from datetime import datetime


/tmp/ipykernel_138329/2347010561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dist_haversine'] = df.apply(lambda row: haversine(row['lat1'], row['lon1'], row['lat2'], row['lon2']), axis=1)


In [13]:
df

,id_rota,nome_usuario,lat2,lon2,elevacao,data,hora,cidade,estado,pais,data_carga_banco,lat1,lon1,dist_euclidiana,dist_manhattan,dist_minkowski,dist_haversine
1,11190523,HYUNDAI_IONIQ_5_2022,-38.473334,144.927550,93.2,2023-11-21,03:58:26,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473332,144.927545,0.005385,0.007,0.005104,0.000489
2,11190523,HYUNDAI_IONIQ_5_2022,-38.473336,144.927554,93.2,2023-11-21,03:58:27,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473334,144.927550,0.004472,0.006,0.004160,0.000413
3,11190523,HYUNDAI_IONIQ_5_2022,-38.473333,144.927554,93.2,2023-11-21,03:58:28,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473336,144.927554,0.003000,0.003,0.003000,0.000334
4,11190523,HYUNDAI_IONIQ_5_2022,-38.473344,144.927568,93.2,2023-11-21,03:58:30,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473333,144.927554,0.017804,0.025,0.015973,0.001727
5,11190523,HYUNDAI_IONIQ_5_2022,-38.473353,144.927583,93.3,2023-11-21,03:58:31,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.473344,144.927568,0.017493,0.024,0.016010,0.001645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7647,11190523,HYUNDAI_IONIQ_5_2022,-38.477056,144.893626,126.8,2023-11-21,06:25:50,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.477057,144.893619,0.007071,0.008,0.007007,0.000619
7648,11190523,HYUNDAI_IONIQ_5_2022,-38.477054,144.893628,126.8,2023-11-21,06:25:51,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.477056,144.893626,0.002828,0.004,0.002520,0.000282
7649,11190523,HYUNDAI_IONIQ_5_2022,-38.477053,144.893628,126.8,2023-11-21,06:25:52,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.477054,144.893628,0.001000,0.001,0.001000,0.000111
7650,11190523,HYUNDAI_IONIQ_5_2022,-38.477053,144.893627,126.8,2023-11-21,06:25:53,Melbourne,AU-VIC,AU,2023-11-21 12:18:07,-38.477053,144.893628,0.001000,0.001,0.001000,0.000087


In [13]:
import pandas as pd
import numpy as np
from datetime import datetime

In [18]:
id_rota = df['id_rota'].iloc[0]
nome_usuario = df['nome_usuario'].iloc[0]
data = df['data'].iloc[0]
hora_inicio = df['hora'].iloc[0]
hora_fim = df['hora'].iloc[-1]  # Última ocorrência de hora
cidade = df['cidade'].iloc[0]
pais = df['pais'].iloc[0]

# Calcular o tempo de viagem
primeira_hora = pd.to_datetime(hora_inicio)
ultima_hora = pd.to_datetime(hora_fim)
if ultima_hora > primeira_hora:
    tempo_viagem = ultima_hora - primeira_hora
else:
    tempo_viagem = primeira_hora - ultima_hora

# Calcular as somas das distâncias
dist_euclidiana = df['dist_euclidiana'].sum()
dist_manhattan = df['dist_manhattan'].sum()
dist_minkowski = df['dist_minkowski'].sum()

# Definir a data de carga do banco como a hora atual
data_carga_banco = datetime.now()

In [22]:
df_distancias = pd.DataFrame({
    'id_rota': [id_rota],
    'nome_usuario': [nome_usuario],
    'data': [data],
    'hora_inicio': [hora_inicio],
    'hora_fim': [hora_fim],
    'tempo_viagem': [tempo_viagem],
    'cidade': [cidade],
    'pais': [pais],
    'dist_euclidiana': [dist_euclidiana],
    'dist_manhattan': [dist_manhattan],
    'dist_minkowski': [dist_minkowski],
    'data_carga_banco': [data_carga_banco]
})

In [23]:
df_distancias


,id_rota,nome_usuario,data,hora_inicio,hora_fim,tempo_viagem,cidade,pais,dist_euclidiana,dist_manhattan,dist_minkowski,data_carga_banco
0,11190523,HYUNDAI_IONIQ_5_2022,2023-11-21,03:58:26,06:25:54,0 days 02:27:28,Melbourne,AU,118.985753,150.16,112.823311,2024-03-19 20:49:59.612378


# 4ª Função - Enriquecendo o DF com informação de pais - estado e cidade - Carregando o DF Enriquecido na camada GOLD

In [ ]:
# pip install minio
# pip install pandas
# pip install numpy

from minio import Minio
from minio.error import S3Error
from io import StringIO, BytesIO
import pandas as pd
from geopy.geocoders import Nominatim

CAMADA_SILVER = 'silver'
CAMADA_GOLD = 'gold'


minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_SILVER) if arquivo_gpx.object_name.endswith(".csv")]

for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:
        
#    obj_rota_csv = minioclient.get_object(CAMADA_SILVER, arquivo_rotas_gpx_csv.object_name)        
#    csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string    
#    arquivo_csv = StringIO(csv_decod)
#    df = pd.read_csv(arquivo_csv, sep=';')

#    batch_size = 1000
#    batch_list = [df[i:i+batch_size].copy() for i in range(0, len(df), batch_size)]
#    geolocator = Nominatim(user_agent="geoapiExercises", timeout=10)
#    processed_batches = []  # Inicialize a lista de lotes processados
#    last_locations = {}

#    for batch_df in batch_list:
#        # Obtenha a primeira e a última linha do lote
#        first_row = batch_df.iloc[0]
#        last_row = batch_df.iloc[-1]

#        first_location_str = f"{first_row['latitude']},{first_row['longitude']}"
#        last_location_str = f"{last_row['latitude']},{last_row['longitude']}"

#        # Verifique se a primeira e a última localização são iguais à última localização verificada
#        if are_locations_equal(first_location_str, last_location_str) and are_locations_equal(first_location_str, last_location):
#            # Se forem iguais, não é necessário consultar o serviço novamente
#            location = last_location
#        else:
#            # Se forem diferentes, consulte o serviço de geocodificação para a última localização
#            location = geolocator.reverse(last_location_str)
#            last_location = location

#        address = location.raw['address']
#        cidade = address.get('county') or address.get('city') or address.get('suburb')
#        estado = address.get('ISO3166-2-lvl4').split('-')[1]
#        pais = address.get('country_code')
                
#        pais = pais.upper()
        
#        df.loc[:, 'cidade'] = cidade
#        df.loc[:, 'estado'] = estado
#        df.loc[:, 'pais'] = pais

#    # Converta o DataFrame enriquecido de volta para CSV
#    enriched_csv = df.to_csv(index=False, sep=';')
#    enriched_csv_bytes = enriched_csv.encode('utf-8')

#    # Crie um buffer de bytes
#    enriched_csv_buffer = BytesIO(enriched_csv_bytes)

#    minioclient.put_object(
#        CAMADA_GOLD,
#        arquivo_rotas_gpx_csv.object_name,  # Use o mesmo nome de arquivo
#        data=enriched_csv_buffer,
#        length=len(enriched_csv_bytes),
#        content_type='application/csv')

In [ ]:
def are_locations_equal(location1, location2):
    return location1 == location2

In [ ]:
# Supondo que 'result' seja o dicionário retornado pela geocodificação
address = result.raw['address']

cidade = None
estado = None
pais = None

# Mapeie os atributos de endereço para os nomes padronizados
for key, aliases in address_mappings.items():
    for alias in aliases:
        if alias in address:
            if key == 'city' or key == 'county':
                cidade = address[alias]                            
            

# Agora você tem 'cidade', 'estado' e 'pais' padronizados independentemente do formato de endereço


In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_SILVER) if arquivo_gpx.object_name.endswith(".csv")]
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:    
    obj_rota_csv = minioclient.get_object(CAMADA_SILVER, arquivo_rotas_gpx_csv.object_name)        
    csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string    
    arquivo_csv = StringIO(csv_decod)
    df = pd.read_csv(arquivo_csv)

In [ ]:
df = pd.read_csv(r'/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO/10125901__Routes_from_sunnypilot_0_9_4_1-release__KIA_EV6_2022__.csv', sep=';')
df

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
Latitude = "40.82486026618991"
Longitude = "-87.99828066563114"
location = geolocator.reverse(f'{Latitude},{Longitude}') 

In [ ]:
cidade = address.get('county') or address.get('city') or address.get('suburb')
estado = address.get('ISO3166-2-lvl4').split('-')[1]
pais = address.get('country_code')
pais


cidade = address.get('county') or address.get('city') or address.get('suburb')
estado = address.get('ISO3166-2-lvl4').split('-')[1]
pais = address.get('country_code')
pais


address = location.raw['address']
cidade = address.get('suburb')
estado = address.get('ISO3166-2-lvl4')
pais = address.get('country_code')

estado = estado[-2:]
pais = pais.upper()

print(f"Cidade: {cidade}")
print(f"Estado (abreviado): {estado[-2:]}")
print(f"País: {pais}")



address = location.raw['address']
cidade = address.get('county')
estado = address.get('ISO3166-2-lvl4')
pais = address.get('country_code')

# 5ª Função - Lendo o bucket Silver e Escrevendo os arquivos no banco de dados Postgres


In [ ]:
##############################################
### IMPORTAÇÃO DAS BIBLIOTECAS NECESSÁRIAS ###
##############################################
# pip install minio
# pip install psycopg2

import psycopg2
from minio import Minio
from io import StringIO, BytesIO
import io
import pandas as pd

####################################
### DEFINIÇÃO DA CAMADA NO MINIO ###
####################################
CAMADA_GOLD = 'gold'
CAMADA_FILES_IN_TABLE = 'files-in-table'

##############################################
### CRIANDO UMA INSTÂNCIA DO CLIENTE MINIO ###
##############################################
minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

#############################################
### ACESSANDO O BANCO DE DADOS POSTGRESQL ###
#############################################
db_config = {
'host': 'localhost',
'database': 'postgres',
'user': 'postgres',
'password': 'postgres',
}

########################################
### COMANDO SQL PARA A OPERAÇÃO COPY ###
########################################
copy_sql = """
    COPY tb_gpx_full (id_rota, nome_usuario, latitude, longitude, elevacao, data_rota, hora_rota, cidade, estado, pais)
    FROM stdin WITH CSV HEADER DELIMITER as ';'
"""

try:
    # Lista todos os arquivos na camada "gold" do Minio que têm extensão .csv
    arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_GOLD) if arquivo_gpx.object_name.endswith(".csv")]
    
    # Verifica se há arquivos no bucket antes de continuar
    if not arquivos_rotas_gpx_csv:
        print("Não existem arquivos CSV no bucket. Nenhuma carga de dados será executada.")

    else:
        # Conexão com o banco de dados PostgreSQL
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        # Itera sobre cada arquivo CSV encontrado no Minio
        for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:
            # Obtém o objeto do arquivo CSV do Minio  
            obj_rota_csv = minioclient.get_object(CAMADA_GOLD, arquivo_rotas_gpx_csv.object_name)            

            # Decodifica os dados do arquivo CSV de bytes para string
            csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string
            arquivo_csv = StringIO(csv_decod)
            df = pd.read_csv(arquivo_csv, sep=';')
            csv_bytes = df.to_csv(index=False,sep=';').encode('utf-8')
            csv_buffer = BytesIO(csv_bytes)
            nome_arquivo = arquivo_rotas_gpx_csv.object_name


            # Usa io.StringIO para criar um objeto de arquivo legível a partir da string CSV
            with io.StringIO(csv_decod) as file:        

                # Executa o comando COPY para inserir os dados no banco de dados PostgreSQL
                cursor.copy_expert(sql=copy_sql, file=file)

            ## Commit para salvar as alterações no banco de dados    
            conn.commit()


            minioclient.put_object( #-->Usdndo o metodo do MinIO responsável por adicionar arquivos no Bucket
                    CAMADA_FILES_IN_TABLE, #--> Nome da camada de destino do arquivo transformado
                    nome_arquivo, #--> Nome do arquivo a ser adicionado na nova camada
                    data=csv_buffer, #--> Objeto csv_buffer que contém os bytes do arquivo CSV.
                    length=len(csv_bytes), #--> Especificando o comprimento dos bytes do arquivo CSV que você está enviando.
                    content_type='application/csv')        

            
            # Após a copia para o bucket de segurança os arquivos são eliminados da camada gold
            minioclient.remove_object(CAMADA_GOLD, arquivo_rotas_gpx_csv.object_name) 

        # Fecha a conexão com o banco de dados PostgreSQL
        conn.close()

except Exception as e:
    # Em caso de erro, imprime a mensagem de erro
    print(f"Erro: {str(e)}")

In [ ]:
from datetime import datetime
data_carga_banco = datetime.now().strftime('%Y-%m-%d')
print(data_carga_banco)


In [ ]:
# Lista todos os arquivos na camada "gold" do Minio que têm extensão .csv
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_GOLD) if arquivo_gpx.object_name.endswith(".csv")]

# Verifica se há arquivos no bucket antes de continuar
if not arquivos_rotas_gpx_csv:
    print("Não existem arquivos CSV no bucket. Nenhuma carga de dados será executada.")
else:
    # Conexão com o banco de dados PostgreSQL
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    # Itera sobre cada arquivo CSV encontrado no Minio
    for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:
        # Obtém o objeto do arquivo CSV do Minio  
        obj_rota_csv = minioclient.get_object(CAMADA_GOLD, arquivo_rotas_gpx_csv.object_name)            

        # Decodifica os dados do arquivo CSV de bytes para string
        csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string
        arquivo_csv = StringIO(csv_decod)
        df = pd.read_csv(arquivo_csv, sep=';')
        csv_bytes = df.to_csv(index=False,sep=';').encode('utf-8')
        csv_buffer = BytesIO(csv_bytes)            
        nome_arquivo = arquivo_rotas_gpx_csv.object_name

        minioclient.put_object( #-->Usdndo o metodo do MinIO responsável por adicionar arquivos no Bucket
                    CAMADA_FILES_IN_TABLE, #--> Nome da camada de destino do arquivo transformado
                    nome_arquivo, #--> Nome do arquivo a ser adicionado na nova camada
                    data=csv_buffer, #--> Objeto csv_buffer que contém os bytes do arquivo CSV.
                    length=len(csv_bytes), #--> Especificando o comprimento dos bytes do arquivo CSV que você está enviando.
                    content_type='application/csv')        

        minioclient.remove_object(CAMADA_GOLD, arquivo_rotas_gpx_csv.object_name) 

In [27]:
import pandas as pd
import numpy as np
import psycopg2
from minio import Minio
import io
from io import StringIO, BytesIO
from datetime import datetime
from scipy.spatial.distance import euclidean, cityblock, minkowski

CAMADA_GOLD = 'gold'
DISTANCIAS = 'distancias'

In [25]:
##############################################
### CRIANDO UMA INSTÂNCIA DO CLIENTE MINIO ###
##############################################
minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

#############################################
### ACESSANDO O BANCO DE DADOS POSTGRESQL ###
#############################################
db_config = {
'host': 'localhost',
'database': 'postgres',
'user': 'postgres',
'password': 'postgres',
}

In [44]:
def calc_dist_euclidiana(row):
    coord1 = (float(row['lon1']), float(row['lat1']))
    coord2 = (float(row['lon2']), float(row['lat2']))
    return euclidean(coord1, coord2) * 1000

def calc_dist_manhattan(row):
    coord1 = (float(row['lon1']), float(row['lat1']))
    coord2 = (float(row['lon2']), float(row['lat2']))
    return cityblock(coord1, coord2)*1000

def calc_dist_minkowski(row, p=3):
    coord1 = (float(row['lon1']), float(row['lat1']))
    coord2 = (float(row['lon2']), float(row['lat2']))
    return minkowski(coord1, coord2, p)*1000

In [46]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_GOLD) if arquivo_gpx.object_name.endswith(".csv")] #--> Listando todos os arquivos da camada bronze do datalake com extensão .csv
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv: #--> Iterando sobre a lista encontrada
    obj_rota_csv = minioclient.get_object(CAMADA_GOLD, arquivo_rotas_gpx_csv.object_name) #--> Obtendo o nome do arquivo de dentro da camada bronze
    csv_decod = obj_rota_csv.data.decode('utf-8') #--> Decodificando o arquivo encontrado para utf-8 - Essa conversão transforma os dados obtidos do arquivo no bucket em bytes
    arquivo_csv = StringIO(csv_decod) #--> Convertendo os bytes em string
    df = pd.read_csv(arquivo_csv, sep=';') #--> Convertendo string para pandas dataframe
    df['lat1'] = df['latitude'].shift() #--> Aqui eu passo a função shift na coluna latitude e já crio uma nova coluna chamada lat 1
    df['lon1'] = df['longitude'].shift() #--> Aqui eu passo a função shift na coluna longitude e já crio uma nova coluna chamada lon 1
    df = df.rename(columns={'latitude': 'lat2', 'longitude': 'lon2'})
    df = df.dropna()

    df['dist_euclidiana'] = df.apply(calc_dist_euclidiana, axis=1)
    df['dist_manhattan'] = df.apply(calc_dist_manhattan, axis=1)
    df['dist_minkowski'] = df.apply(calc_dist_minkowski, axis=1)


#     # Obtendo as informações isoladas
#     id_rota = df['id_rota'].iloc[0]
#     nome_usuario = df['nome_usuario'].iloc[0]
#     data = df['data'].iloc[0]
#     hora_inicio = df['hora'].iloc[0]
#     hora_fim = df['hora'].iloc[-1]  # Última ocorrência de hora
#     cidade = df['cidade'].iloc[0]
#     pais = df['pais'].iloc[0]
#     # Calcular o tempo de viagem
#     primeira_hora = pd.to_datetime(hora_inicio)
#     ultima_hora = pd.to_datetime(hora_fim)
#     if ultima_hora > primeira_hora:
#         tempo_viagem = ultima_hora - primeira_hora
#     else:
#         tempo_viagem = primeira_hora - ultima_hora
#     # Calcular as somas das distâncias
#     dist_euclidiana = df['dist_euclidiana'].sum()
#     dist_manhattan = df['dist_manhattan'].sum()
#     dist_minkowski = df['dist_minkowski'].sum()
#     # Definir a data de carga do banco como a hora atual
#     data_carga_banco = datetime.now()
#     # Criando o dataframe
#     df_distancias = pd.DataFrame({
#         'id_rota': [id_rota],
#         'nome_usuario': [nome_usuario],
#         'data': [data],
#         'hora_inicio': [hora_inicio],
#         'hora_fim': [hora_fim],
#         'tempo_viagem': [tempo_viagem],
#         'cidade': [cidade],
#         'pais': [pais],
#         'dist_euclidiana': [dist_euclidiana],
#         'dist_manhattan': [dist_manhattan],
#         'dist_minkowski': [dist_minkowski],
#         'data_carga_banco': [data_carga_banco]
#     })

#     # Converta o DataFrame enriquecido de volta para CSV
#     distancias_csv = df_distancias.to_csv(index=False, sep=';')
#     distancias_csv_bytes = distancias_csv.encode('utf-8')

# # Crie um buffer de bytes
#     distancias_csv_buffer = BytesIO(distancias_csv_bytes)

#     minioclient.put_object(
#         DISTANCIAS,
#         arquivo_rotas_gpx_csv.object_name,  # Use o mesmo nome de arquivo
#         data=distancias_csv_buffer,
#         length=len(distancias_csv_bytes),
#         content_type='application/csv')

In [63]:
from dotenv import dotenv_values
config = dotenv_values("/home/thiago/tcc_ufrj/scripts_finalizados/.env")

CREDENTIAL = config.get("TOKEN_API")
type(CREDENTIAL)


str

In [61]:
print(CREDENTIAL)

None


In [ ]:
token_api = '5b3ce3597851110001cf624838eb860780704eca99c41867a83c9f6b'

In [58]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_GOLD) if arquivo_gpx.object_name.endswith(".csv")] #--> Listando todos os arquivos da camada bronze do datalake com extensão .csv
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv: #--> Iterando sobre a lista encontrada
    obj_rota_csv = minioclient.get_object(CAMADA_GOLD, arquivo_rotas_gpx_csv.object_name) #--> Obtendo o nome do arquivo de dentro da camada bronze
    csv_decod = obj_rota_csv.data.decode('utf-8') #--> Decodificando o arquivo encontrado para utf-8 - Essa conversão transforma os dados obtidos do arquivo no bucket em bytes
    arquivo_csv = StringIO(csv_decod) #--> Convertendo os bytes em string
    df = pd.read_csv(arquivo_csv, sep=';') #--> Convertendo string para pandas dataframe

    #df = pd.read_csv('11190523__routes__HYUNDAI_IONIQ_5_2022.csv', sep=';')
    df['lat1'] = df['latitude'].shift() #--> Aqui eu passo a função shift na coluna latitude e já crio uma nova coluna chamada lat 1
    df['lon1'] = df['longitude'].shift() #--> Aqui eu passo a função shift na coluna longitude e já crio uma nova coluna chamada lon 1
    df = df.rename(columns={'latitude': 'lat2', 'longitude': 'lon2'})
    df = df.dropna(subset=['lat1', 'lon1'])


    df['dist_euclidiana'] = df.apply(calc_dist_euclidiana, axis=1)
    df['dist_manhattan'] = df.apply(calc_dist_manhattan, axis=1)
    df['dist_minkowski'] = df.apply(calc_dist_minkowski, axis=1)


    # Obtendo as informações isoladas
    id_rota = df['id_rota'].iloc[0]
    nome_usuario = df['nome_usuario'].iloc[0]
    data_viagem = df['data'].iloc[0]
    hora_inicio = df['hora'].iloc[0]
    hora_fim = df['hora'].iloc[-1]  # Última ocorrência de hora
    cidade = df['cidade'].iloc[0]
    pais = df['pais'].iloc[0]
    # Calcular o tempo de viagem
    primeira_hora = pd.to_datetime(hora_inicio)
    ultima_hora = pd.to_datetime(hora_fim)
    if ultima_hora > primeira_hora:
        tempo_viagem = ultima_hora - primeira_hora
    else:
        tempo_viagem = primeira_hora - ultima_hora
    # Calcular as somas das distâncias
    dist_euclidiana = df['dist_euclidiana'].sum()
    dist_manhattan = df['dist_manhattan'].sum()
    dist_minkowski = df['dist_minkowski'].sum()
    # Definir a data de carga do banco como a hora atual
    data_carga_banco = datetime.now()
    # Criando o dataframe
    df_distancias = pd.DataFrame({
        'id_rota': [id_rota],
        'nome_usuario': [nome_usuario],
        'data_viagem': [data_viagem],
        'hora_inicio': [hora_inicio],
        'hora_fim': [hora_fim],
        'tempo_viagem': [tempo_viagem],
        'cidade': [cidade],
        'pais': [pais],
        'dist_euclidiana': [dist_euclidiana],
        'dist_manhattan': [dist_manhattan],
        'dist_minkowski': [dist_minkowski],
        'data_carga_banco': [data_carga_banco]
    })


# Converta o DataFrame enriquecido de volta para CSV
    distancias_csv = df_distancias.to_csv(index=False, sep=';')
    distancias_csv_bytes = distancias_csv.encode('utf-8')

# Crie um buffer de bytes
    distancias_csv_buffer = BytesIO(distancias_csv_bytes)

    minioclient.put_object(
        DISTANCIAS,
        arquivo_rotas_gpx_csv.object_name,  # Use o mesmo nome de arquivo
        data=distancias_csv_buffer,
        length=len(distancias_csv_bytes),
        content_type='application/csv')
 

# Explorando o geopy.geocoders

In [ ]:
# pip install openrouteservice 
# pip install folium 
# pip install pyrosm

# https://openrouteservice.org/

import psycopg2
from minio import Minio
import io
from io import StringIO, BytesIO
import pandas as pd
#import openrouteservice
#import ast
#from datetime import datetime, timedelta
import requests


import psycopg2
import pandas as pd
import openrouteservice
import ast
from datetime import datetime, timedelta
from geopy.distance import geodesic as GD

api = 'api_route'
token_api = '5b3ce3597851110001cf624838eb860780704eca99c41867a83c9f6b'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)


db_config = {
'host': 'localhost',
'database': 'postgres',
'user': 'postgres',
'password': 'postgres',
}

CAMADA_GOLD = 'gold'
CAMADA_FILES_IN_TABLE = 'files-in-table'

In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_GOLD) if arquivo_gpx.object_name.endswith(".csv")] #--> Listando todos os arquivos da camada bronze do datalake com extensão .csv
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv: #--> Iterando sobre a lista encontrada
    try:    
        obj_rota_csv = minioclient.get_object(CAMADA_GOLD, arquivo_rotas_gpx_csv.object_name) #--> Obtendo o nome do arquivo de dentro da camada bronze
        csv_decod = obj_rota_csv.data.decode('utf-8') #--> Decodificando o arquivo encontrado para utf-8 - Essa conversão transforma os dados obtidos do arquivo no bucket em bytes
        arquivo_csv = StringIO(csv_decod) #--> Convertendo os bytes em string
        df = pd.read_csv(arquivo_csv, sep=';') #--> Convertendo string para pandas dataframe        
        
        id_rota = df.loc[df.index[0], 'id_rota']
        nome_usuario = df.loc[df.index[0], 'nome_usuario']
        data_inicio_rota = df.loc[df.index[0], 'data']
        data_fim_rota = df.loc[df.index[-1], 'data']
        inicio_rota = df.loc[df.index[0], 'hora']
        fim_rota = df.loc[df.index[-1], 'hora']
        latitude_inicial = df.loc[df.index[0], 'latitude']
        longitude_inicial = df.loc[df.index[0], 'longitude']
        latitude_final = df.loc[df.index[-1], 'latitude']
        longitude_final = df.loc[df.index[-1], 'longitude']
        cidade = df.loc[df.index[0], 'cidade']
        estado = df.loc[df.index[0], 'estado']
        pais = df.loc[df.index[0], 'pais']
        id_unico = f'{id_rota}__{data_inicio_rota}__{inicio_rota}__{nome_usuario}'
        data_carga_banco = df.loc[df.index[0], 'data_carga_banco']
        longitude_inicial_float = float(longitude_inicial)
        latitude_inicial_float = float(latitude_inicial)    
        longitude_final_float = float(longitude_final)
        latitude_final_float = float(latitude_final)    
        #body = {"coordinates":[[8.681495,49.41461],[8.687872,49.420318]],"radiuses":"-1"}
        body = {"coordinates": [[longitude_inicial_float, latitude_inicial_float],[longitude_final_float, latitude_final_float]],"radiuses": "-1"}

        headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': token_api,
            'Content-Type': 'application/json; charset=utf-8'
        }
    
        call = requests.post('https://api.openrouteservice.org/v2/directions/driving-car', json=body, headers=headers)
        response = eval(call.text)
        duration = response['routes'][0]['summary']['duration']
        distance = response['routes'][0]['summary']['distance']
        distancia_real = distance / 1000

#        conn = psycopg2.connect(**db_config)
#        cursor = conn.cursor()
#        insert = f'''
#            insert into tb_distancias_percorridas_api (id_unico,nome_usuario,data_inicio_rota,data_fim_rota,hora_inicio_rota,hora_fim_rota,latitude_inicial,longitude_inicial,latitude_final,longitude_final,cidade,estado,pais,distancia_real_km_api,data_carga_banco)
#            values (
#                '{id_unico}',
#                '{nome_usuario}',
#                '{data_inicio_rota}',
#                '{data_fim_rota}',
#                '{inicio_rota}',
#                '{fim_rota}',
#                '{latitude_inicial}',
#                '{longitude_inicial}',
#                '{latitude_final}',
#                '{longitude_final}',
#                '{cidade}',
#                '{estado}',
#                '{pais}',              
#                '{distancia_real}',
#                '{data_carga_banco}'
#                )'''
#        cursor.execute(insert)
#        conn.commit()
#        conn.close()
#        print(insert)
    except Exception as e:        
        print(f"Erro: {str(e)} no id {id_unico}")
        continue

In [ ]:
body = {"coordinates": [[0.5630799941718578, 48.37511999532581],[0.5221799947321415, 48.324159951880574]],"radiuses": "-1"}

headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': token_api,
            'Content-Type': 'application/json; charset=utf-8'
        }
    
call = requests.post('https://api.openrouteservice.org/v2/directions/driving-car', json=body, headers=headers)
response = eval(call.text)

#48.37511999532581	0.5630799941718578	48.324159951880574	0.5221799947321415

In [ ]:
response

In [ ]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

consulta_veiculos = """
SELECT
	b.id_unico,	
    b.nome_usuario,
    b.data_rota,
    b.inicio_rota,
    b.fim_rota,
    b.tempo_viagem,
    b.first_lat,
	b.first_long,
	b.last_lat,
	b.last_long,
    b.cidade,
    b.estado,
    b.pais,
    b.dist_linear_km,
    b.dist_manhattan_km,	
	DATE(b.carga_banco)
FROM (
    select
		concat(a.id_rota,'__',a.data_rota,'__',a.inicio_rota,'__',a.nome_usuario) as id_unico,        
        a.nome_usuario,
        a.data_rota,
        a.inicio_rota,
        a.fim_rota,
				case when a.fim_rota > a.inicio_rota then a.fim_rota - a.inicio_rota
        	 when a.inicio_rota > a.fim_rota then a.inicio_rota - a.fim_rota end as tempo_viagem,
        a.first_lat,
        a.first_long,
        a.last_lat,
        a.last_long,
        a.cidade,
        a.estado,
        a.pais,
        st_distance(
            st_geographyfromtext(((('POINT('::text || a.last_long::text) || ' '::text) || a.last_lat::text) || ')'::text),
            st_geographyfromtext(((('POINT('::text || a.first_long::text) || ' '::text) || a.first_lat::text) || ')'::text)
        ) / 1000.0::double precision AS dist_linear_km,
        abs(cast(a.last_long as numeric) - cast(a.first_long as NUMERIC)) + abs(cast(a.last_lat as NUMERIC)- cast(a.first_lat as NUMERIC)) * 1000.0::double precision AS dist_manhattan_km,
        a.carga_banco
    FROM (
        SELECT DISTINCT
            tb_gpx_full.id_rota,
            tb_gpx_full.nome_usuario,
            tb_gpx_full.data_rota, 
            tb_gpx_full.cidade,
            tb_gpx_full.estado,
            tb_gpx_full.pais,
            first_value(tb_gpx_full.hora_rota) OVER (PARTITION BY tb_gpx_full.id_rota ORDER BY tb_gpx_full.id_rota)::time without time zone AS inicio_rota,
            last_value(tb_gpx_full.hora_rota) OVER (PARTITION BY tb_gpx_full.id_rota ORDER BY tb_gpx_full.id_rota)::time without time zone AS fim_rota,
            first_value(tb_gpx_full.longitude) OVER (PARTITION BY tb_gpx_full.id_rota ORDER BY tb_gpx_full.id_rota) AS first_long,
            last_value(tb_gpx_full.longitude) OVER (PARTITION BY tb_gpx_full.id_rota ORDER BY tb_gpx_full.id_rota) AS last_long,
            first_value(tb_gpx_full.latitude) OVER (PARTITION BY tb_gpx_full.id_rota ORDER BY tb_gpx_full.id_rota) AS first_lat,
            last_value(tb_gpx_full.latitude) OVER (PARTITION BY tb_gpx_full.id_rota ORDER BY tb_gpx_full.id_rota) AS last_lat,
            carga_banco::timestamp
        FROM tb_gpx_full
    ) a
) b
where 	b.data_rota is not null 
and 	b.inicio_rota is not null 
and 	b.tempo_viagem is not null
and     DATE(b.carga_banco) = date(now())

ORDER BY b.nome_usuario, b.data_rota, b.dist_manhattan_km desc ;


"""

cursor.execute(consulta_veiculos)
resultados_dist_veiculos = cursor.fetchall()
conn.close()
df = pd.DataFrame(resultados_dist_veiculos, columns=[desc[0] for desc in cursor.description])


In [ ]:
df

In [ ]:
# id_unico = df.loc[df.index[0], 'id_unico']
# nome_usuario = df.loc[df.index[0], 'nome_usuario']

#data_inicio_rota = datetime.strptime(df.loc[df.index[0], 'data'], '%Y-%m-%d').date()
#data_fim_rota = datetime.strptime(df.loc[df.index[-1], 'data'], '%Y-%m-%d').date()
#inicio_rota = datetime.strptime(df.loc[df.index[0], 'hora'], '%H:%M:%S').time()
#inicio_rota = datetime.combine(datetime.min, inicio_rota)
#fim_rota = datetime.strptime(df.loc[df.index[-1], 'hora'], '%H:%M:%S').time()
#fim_rota = datetime.combine(datetime.min, fim_rota)
#latitude_inicial = df.loc[df.index[0], 'latitude']
#longitude_inicial = df.loc[df.index[0], 'longitude']
#latitude_final = df.loc[df.index[-1], 'latitude']
#longitude_final = df.loc[df.index[-1], 'longitude']
#cidade = df.loc[df.index[0], 'cidade']
#estado = df.loc[df.index[0], 'estado']
#pais = df.loc[df.index[0], 'pais']
#if data_inicio_rota == data_fim_rota:
#    data_rota = data_inicio_rota
#    if inicio_rota > fim_rota:
#        tempo_viagem_real = inicio_rota - fim_rota
#    elif fim_rota > inicio_rota:
#        tempo_viagem_real = fim_rota - inicio_rota
#    else:
#        tempo_viagem_real = '0'
#elif data_inicio_rota < data_fim_rota:
#    data_rota = data_inicio_rota
#    tempo_viagem_real = fim_rota - inicio_rota
#    tempo_viagem_real = str(tempo_viagem_real)
#    tempo_viagem_real = tempo_viagem_real.split(', ')
#    tempo_viagem_real = tempo_viagem_real[1]
#else:
#    tempo_viagem_real = '0'
#horas, minutos, segundos = map(int, tempo_viagem_real.split(':'))
#tempo_viagem_formatado = f'{horas:02d}:{minutos:02d}:{segundos:02d}'

#dist_linear = GD(lat_long_ini,lat_long_fin).km
#

#dist_manhattan_km = 


#lat_long_ini = float(first_lat),float(first_long)
#lat_long_fin = float(last_lat),float(last_long)

id_unicos = df['id_unico'].to_list()
firsts_lat = df['first_lat'].to_list()
firsts_long = df['first_long'].to_list()
lasts_lat = df['last_lat'].to_list()
lasts_long = df['last_long'].to_list()



In [ ]:
try:
    for first_long, first_lat, last_long, last_lat,id_unico in zip(firsts_long,firsts_lat,lasts_long,lasts_lat,id_unicos):        
        coords = f'(({first_long},{first_lat}),({last_long},{last_lat}))'
        coords = ast.literal_eval(coords)
        client = openrouteservice.Client(key=token_key) # Specify your personal API key
        response = client.directions(coords)
        routes = response['routes']
        summary = routes[0]['summary']
        distancia_real = summary['distance']
        distancia_real = distancia_real / 1000
except Exception as e:
    # Em caso de erro, imprime a mensagem de erro
    print(f"Erro: {str(e)} no id {id_unico}")
    continue  

In [ ]:
coords = f'(({first_long},{first_lat}),({last_long},{last_lat}))'
coords = ast.literal_eval(coords)
coords

In [ ]:
client = openrouteservice.Client(key=token_key) # Specify your personal API key
response = client.directions(coords)

In [ ]:
response

In [ ]:
routes = response['routes']
summary = routes[0]['summary']
distancia_real = summary['distance']
distancia_real = distancia_real / 1000
tempo_viagem_estimado = summary['duration']
horas, segundos_restantes = divmod(tempo_viagem_estimado, 3600)
minutos, segundos = divmod(segundos_restantes, 60)
tempo_viagem_estimado_formatado = '{:02}:{:02}:{:02}'.format(int(horas), int(minutos), int(segundos))

In [ ]:
tempo_viagem_estimado_formatado

In [ ]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
insert = f'''
insert into tb_distancias_percorridas (id_unico,nome_usuario,data_rota,inicio_rota,fim_rota,tempo_viagem_real,tempo_viagem_estimado ,cidade,estado,pais,dist_real)
values (
        '{id_unico}',
        '{nome_usuario}',
        '{data_rota}',
        '{inicio_rota.strftime('%H:%M:%S')}',
        '{fim_rota.strftime('%H:%M:%S')}',
        '{tempo_viagem_formatado}',
        '{tempo_viagem_estimado_formatado}',
        '{cidade}',
        '{estado}',
        '{pais}',
        '{distancia_real}'
        )
'''
print(insert)

In [ ]:
cursor.execute(insert)
conn.commit()
conn.close()

In [ ]:
from geopy.geocoders import Nominatim
from geopy import distance

geolocator = Nominatim(user_agent="geoapiExercises")
Latitude = "45.24559097710776"
Longitude = "-122.79781984274484"
location = geolocator.reverse(f'{Latitude},{Longitude}') 
print(location)

In [ ]:
address = location.raw['address']
address

In [ ]:
address = location.raw['address']
city = address.get('city', '')
state = address.get('state', '')
country = address.get('country', '')

In [ ]:
address

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
Latitude = "45.24559097710776"
Longitude = "-122.79781984274484"
location = geolocator.reverse(f'{Latitude},{Longitude}') 
print(location)

address = location.raw['address']
cidade = address.get('suburb')
estado = address.get('ISO3166-2-lvl4')
pais = address.get('country_code')

In [ ]:
address

In [ ]:
print("Cidade:", cidade)
print("Estado (abreviado):", estado)
print("País:", pais)

In [ ]:
db_config = {
    'host': 'localhost',
    'database': 'gold-saint',
    'user': 'postgres',
    'password': 'postgres',
}


In [ ]:
sql_query = "SELECT * FROM tb_gpx_full"

In [ ]:
conn = psycopg2.connect(**db_config)

df_pandas = pd.read_sql_query(sql_query, conn)

conn.close()

df_pandas


In [ ]:
def obter_info_localizacao(df):
    geolocator = Nominatim(user_agent="geoapiExercises")
    latitudes = df['latitude'].astype(str)
    longitudes = df['longitude'].astype(str)

    def obter_localizacao(latitude, longitude):
        location = geolocator.reverse(f'{latitude},{longitude}')
        address = location.raw['address']
        cidade = address.get('suburb')
        estado = address.get('state')
        pais = address.get('country_code')
        return cidade, estado, pais
    df[['cidade', 'estado', 'pais']] = zip(*[obter_localizacao(lat, lon) for lat, lon in zip(latitudes, longitudes)])
    return df

In [ ]:
df_resultante = obter_info_localizacao(df_pandas)
df_resultante

# WebScraping para obter o arquivo com o preco médio de combustível -- Pendente finalizar o webscrapping da tabela da ANP

### Importação das bibliotecas necessárias para a raspagem de dados.

In [ ]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import os
import pandas as pd
from minio import Minio #--> O módulo minio é usado para interagir com um servidor MinIO
from minio.error import S3Error #--> O módulo S3Error é uma exceção específica do MinIO para exibição de forma semelhante ao Amazon S3
from io import StringIO, BytesIO
import psycopg2
import io


db_config = {
'host': 'localhost',
'database': 'postgres',
'user': 'postgres',
'password': 'postgres',
}


URL_ANP_PRECO_MEDIO_SEMANAL = 'https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/levantamento-de-precos-de-combustiveis-ultimas-semanas-pesquisadas' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
DOWNLOADS = '/home/thiago/Downloads/'
DADOS_COMBUSTIVEL = '/home/thiago/tcc_ufrj/DADOS_COMBUSTIVEL/'
BUCKET_DADOS_COMBUSTIVEL = 'dados-combustivel'



minioclient = Minio('localhost:9000', #--> O cliente é configurado para se conectar a um servidor MinIO local usando as credenciais fornecidas
    access_key='minioadmin', #--> A chave de acesso = usuário
    secret_key='minioadmin', #--> A chave secreta = Senha 
    secure=False) #--> Sem usar conexão segura (HTTPS).


# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

# navegador = webdriver.Chrome(service=servico)
# navegador.get(URL_ANP_PRECO_MEDIO_SEMANAL)

# conteudo_da_pagina = navegador.page_source
# site = BeautifulSoup(conteudo_da_pagina, 'html.parser')

In [ ]:
## Transformando o xlsx em um Pandas DataFrame 
arquivos_combustivel = [arquivo for arquivo in os.listdir(DADOS_COMBUSTIVEL) if arquivo.endswith(".xlsx")] #--> Listando todos os arquivos da pasta pré-processamento com extensão .gpx

for arquivo_combustivel in arquivos_combustivel:
    caminho_arquivo = os.path.join(DADOS_COMBUSTIVEL, arquivo_combustivel) #--> Criando o caminho completo para o arquivo .gpx        
    df = pd.read_excel(caminho_arquivo, sheet_name='CAPITAIS', skiprows=9)
    df.to_csv(f'{DADOS_COMBUSTIVEL}PRECO_NACIONAL.csv', sep=';', encoding='utf8', index=False)
    df.head()
# df.dtypes
# df_rj = df[df['MUNICÍPIO'] == 'RIO DE JANEIRO']
# df_rj.to_csv(f'{DADOS_COMBUSTIVEL}PRECO_RJ.csv', sep=';', encoding='utf8', index=False)
# df_rj

In [ ]:
df.head()

In [ ]:
## Enviando CSV para o Datalake
arquivos_para_datalake = [arquivo for arquivo in os.listdir(DADOS_COMBUSTIVEL) if arquivo.endswith(".csv")] #--> Listando todos os arquivos da pasta pré-processamento com extensão .csv
for nome_arquivo in arquivos_para_datalake: #--> Iterando sobre cada item da lista
    caminho_pre_proc = os.path.join(DADOS_COMBUSTIVEL, nome_arquivo) #--> Criando o caminho completo para o arquivo .csv    
    if os.path.isfile(caminho_pre_proc): #--> Verificando se o caminho especificado está apontando para um arquivo válido no sistema de arquivos.
        try:
            minioclient.fput_object(BUCKET_DADOS_COMBUSTIVEL, nome_arquivo, caminho_pre_proc) #--> Usando o cliente Minio para enviar o arquivo da pasta de pré processamento para o bucket especificado (CAMADA_BRONZE)
            #print(f"Arquivo {nome_arquivo} enviado com sucesso para o bucket.") #--> Exibindo a mensagem de sucesso após o upload dos arquivos para o bucket.
            os.remove(caminho_pre_proc) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta DADOS_COMBUSTIVEL
        except S3Error as e: #--> Capturando qualquer erro que porventura ocorra
            print(f"Erro ao enviar o arquivo: {nome_arquivo} -> Erro: {e}") #--> Exibindo o erro


In [ ]:
## Conexão e escrita no banco de dados do arquivo CSV obtido do Bucket
try:
    # Lista todos os arquivos na camada "gold" do Minio que têm extensão .csv
    arquivos_preco_combustivel = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(BUCKET_DADOS_COMBUSTIVEL) if arquivo_gpx.object_name.endswith(".csv")]
    
    # Verifica se há arquivos no bucket antes de continuar
    if not arquivos_preco_combustivel:
        print("Não existem arquivos CSV no bucket. Nenhuma carga de dados será executada.")

    else:
        # Conexão com o banco de dados PostgreSQL
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        #truncate = """ truncate table preco_combustivel_semanal; """
        #cursor.execute(truncate)

        copy_sql = """
            COPY preco_combustivel_semanal (data_inicial,data_final,estado,municipio,produto,numero_de_postos_pesquisados,unidade_de_medida,preco_medio_revenda,desvio_padrao_revenda,preco_minimo_revenda,preco_maximo_revenda,coef_de_variacao_revenda)
            FROM stdin WITH CSV HEADER DELIMITER as ';'
        """

        # Itera sobre cada arquivo CSV encontrado no Minio
        for arquivo_preco_combustivel in arquivos_preco_combustivel:
            # Obtém o objeto do arquivo CSV do Minio  
            obj_preco = minioclient.get_object(BUCKET_DADOS_COMBUSTIVEL, arquivo_preco_combustivel.object_name)            

            # Decodifica os dados do arquivo CSV de bytes para string
            csv_decod = obj_preco.data.decode('utf-8')  # Convertendo bytes para string
            arquivo_csv = StringIO(csv_decod)
            df = pd.read_csv(arquivo_csv, sep=';')
            csv_bytes = df.to_csv(index=False,sep=';').encode('utf-8')
            csv_buffer = BytesIO(csv_bytes)
            nome_arquivo = arquivo_preco_combustivel.object_name


            # Usa io.StringIO para criar um objeto de arquivo legível a partir da string CSV
            with io.StringIO(csv_decod) as file:        

                # Executa o comando COPY para inserir os dados no banco de dados PostgreSQL
                cursor.copy_expert(sql=copy_sql, file=file)

            ## Commit para salvar as alterações no banco de dados    
            conn.commit()

            # Após a copia para o bucket de segurança os arquivos são eliminados da camada gold
            minioclient.remove_object(BUCKET_DADOS_COMBUSTIVEL, arquivo_preco_combustivel.object_name) 

        # Fecha a conexão com o banco de dados PostgreSQL
        conn.close()

except Exception as e:
    # Em caso de erro, imprime a mensagem de erro
    print(f"Erro: {str(e)}")

## API para criar tabela com preco de combustível -- API para cirar um DF com o consumo de cada veículo

In [ ]:
#Para o combustivel - fazer um webscraping da tabela da anp
# https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-de-distribuicao-de-combustiveis

In [ ]:
# Abrindo novamente a conexão com o banco para selecionar da tabela gpx full somente o nome a marca e o modelo dos veículos
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
import requests

consulta_usuarios_gpx_full = """
select distinct
split_part(LOWER(nome_usuario),'_',1) as marca,
split_part(LOWER(nome_usuario),'_',2) as modelo
from tb_gpx_full

"""
cursor.execute(consulta_usuarios_gpx_full)
resultados_usuarios_gpx_full = cursor.fetchall()
conn.close()
df_usuarios_full = pd.DataFrame(resultados_usuarios_gpx_full, columns=[desc[0] for desc in cursor.description])
df_usuarios_full.head()
 
df_marcas = df_usuarios_full['marca'].tolist()
df_modelos = df_usuarios_full['modelo'].tolist()

In [ ]:
#df_marcas 
#df_modelos

In [ ]:
df_marca = 'ram'
df_modelo = '1500'
df_carros = pd.DataFrame()
api_url = 'https://api.api-ninjas.com/v1/cars'
api_key = '/DHg+PPb3h7gYITeEup54w==KXt6OHpmw3zMNgfE'
params = {'make': df_marca, 'model': df_modelo}
headers = {'X-Api-Key': api_key}
response = requests.get(api_url, params=params, headers=headers)
        
if response.status_code == requests.codes.ok:            
    data = response.json()
    df_car = pd.DataFrame(data)
    df_carros = pd.concat([df_carros, df_car], ignore_index=True)

else:
    print("Error:", response.status_code, response.text)

In [ ]:
# Consumindo a API para criar um DF com o consumo em litros por KM dos veículos
df_carros = pd.DataFrame()
api_url = 'https://api.api-ninjas.com/v1/cars'
api_key = '/DHg+PPb3h7gYITeEup54w==KXt6OHpmw3zMNgfE'

for df_marca, df_modelo in  zip(df_marcas, df_modelos):
    params = {'make': df_marca, 'model': df_modelo}
    headers = {'X-Api-Key': api_key}
    response = requests.get(api_url, params=params, headers=headers)
        
    if response.status_code == requests.codes.ok:            
        data = response.json()
        df_car = pd.DataFrame(data)
        df_carros = pd.concat([df_carros, df_car], ignore_index=True)

    else:
        print("Error:", response.status_code, response.text)

In [ ]:
df_carros.head()

In [ ]:
df_carros['city_km/l'] = (df_carros['city_mpg'] * 1.609344) / 3.785411784
df_carros['highway_km/l'] = (df_carros['highway_mpg'] * 1.609344) / 3.785411784
df_carros.drop('city_mpg', axis=1, inplace=True)
df_carros.drop('highway_mpg', axis=1, inplace=True)
df_carros.drop('combination_mpg', axis=1, inplace=True)

df_carros.to_csv(f'{DADOS_COMBUSTIVEL}consumo_veiculos.csv', sep=';', encoding='utf8', index=False)
#print(df_carros.dtypes)

In [ ]:
df_carros

In [ ]:
## Enviando CSV para o Datalake
arquivos_para_datalake = [arquivo for arquivo in os.listdir(DADOS_COMBUSTIVEL) if arquivo.endswith(".csv")] #--> Listando todos os arquivos da pasta pré-processamento com extensão .csv
arquivos_para_datalake
for nome_arquivo in arquivos_para_datalake: #--> Iterando sobre cada item da lista
    caminho_pre_proc = os.path.join(DADOS_COMBUSTIVEL, nome_arquivo) #--> Criando o caminho completo para o arquivo .csv    
    if os.path.isfile(caminho_pre_proc): #--> Verificando se o caminho especificado está apontando para um arquivo válido no sistema de arquivos.
        try:
            minioclient.fput_object(BUCKET_DADOS_COMBUSTIVEL, nome_arquivo, caminho_pre_proc) #--> Usando o cliente Minio para enviar o arquivo da pasta de pré processamento para o bucket especificado (CAMADA_BRONZE)
            #print(f"Arquivo {nome_arquivo} enviado com sucesso para o bucket.") #--> Exibindo a mensagem de sucesso após o upload dos arquivos para o bucket.
            os.remove(caminho_pre_proc) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta DADOS_COMBUSTIVEL
        except S3Error as e: #--> Capturando qualquer erro que porventura ocorra
            print(f"Erro ao enviar o arquivo: {nome_arquivo} -> Erro: {e}") #--> Exibindo o erro


In [ ]:
## Conexão e escrita no banco de dados do arquivo CSV obtido do Bucket
try:
    # Lista todos os arquivos na camada "gold" do Minio que têm extensão .csv
    arquivo_consumo_veiculos = [arquivo for arquivo in minioclient.list_objects(BUCKET_DADOS_COMBUSTIVEL) if arquivo.object_name.endswith(".csv")]
    
    # Verifica se há arquivos no bucket antes de continuar
    if not arquivo_consumo_veiculos:
        print("Não existem arquivos CSV no bucket. Nenhuma carga de dados será executada.")

    else:
        # Conexão com o banco de dados PostgreSQL
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        truncate = """ truncate table tb_consumo_veiculos; """
        cursor.execute(truncate)

        copy_sql = """
            COPY tb_consumo_veiculos (classe,drive,fuel_type,make,model,transmission,years,cylinders,displacement,city_km_l,highway_km_l)
            FROM stdin WITH CSV HEADER DELIMITER as ';'
        """

        # Itera sobre cada arquivo CSV encontrado no Minio
        for arquivo_consumo_veiculo in arquivo_consumo_veiculos:
            # Obtém o objeto do arquivo CSV do Minio  
            obj_preco = minioclient.get_object(BUCKET_DADOS_COMBUSTIVEL, arquivo_consumo_veiculo.object_name)            

            # Decodifica os dados do arquivo CSV de bytes para string
            csv_decod = obj_preco.data.decode('utf-8')  # Convertendo bytes para string
            arquivo_csv = StringIO(csv_decod)
            df = pd.read_csv(arquivo_csv, sep=';')
            csv_bytes = df.to_csv(index=False,sep=';').encode('utf-8')
            csv_buffer = BytesIO(csv_bytes)
            nome_arquivo = arquivo_consumo_veiculo.object_name


            # Usa io.StringIO para criar um objeto de arquivo legível a partir da string CSV
            with io.StringIO(csv_decod) as file:        

                # Executa o comando COPY para inserir os dados no banco de dados PostgreSQL
                cursor.copy_expert(sql=copy_sql, file=file)

            ## Commit para salvar as alterações no banco de dados    
            conn.commit()

            # Após a copia para o bucket de segurança os arquivos são eliminados da camada gold
            minioclient.remove_object(BUCKET_DADOS_COMBUSTIVEL, arquivo_consumo_veiculo.object_name) 

        # Fecha a conexão com o banco de dados PostgreSQL
        conn.close()

except Exception as e:
    # Em caso de erro, imprime a mensagem de erro
    print(f"Erro: {str(e)}")

In [ ]:
import requests

make = df_marca
model = df_modelo
api_url = 'https://api.api-ninjas.com/v1/cars?model={}'.format(make,model)
api_key = '/DHg+PPb3h7gYITeEup54w==KXt6OHpmw3zMNgfE'
response = requests.get(api_url, headers={'X-Api-Key': api_key})
if response.status_code == requests.codes.ok:    
    #df = pd.DataFrame(response.text)
    data = response.json()
    df_car = pd.DataFrame(data)
else:
    print("Error:", response.status_code, response.text)



In [ ]:
df_car

In [ ]:
api_url = 'https://api.api-ninjas.com/v1/cars?model={}'.format('skoda','kodiaq')
response = requests.get(api_url, headers={'X-Api-Key': '/DHg+PPb3h7gYITeEup54w==KXt6OHpmw3zMNgfE'})
if response.status_code == requests.codes.ok:       
    data = response.json()
    df_car = pd.DataFrame(data)

df_car   

In [ ]:
import requests
import pandas as pd

make = df_marca  # Substitua df_marca pelo valor desejado
model = df_modelo  # Substitua df_modelo pelo valor desejado
api_url = 'https://api.api-ninjas.com/v1/cars?model={}'.format(make,model)
headers = {'X-Api-Key': '/DHg+PPb3h7gYITeEup54w==KXt6OHpmw3zMNgfE'}

try:
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        data = response.json()  # Converte o JSON da resposta em um objeto Python
        df = pd.DataFrame(data)  # Cria um DataFrame com base nos dados
        print(df)
    else:
        print("Erro:", response.status_code, response.text)
except Exception as e:
    print("Erro ao fazer a solicitação à API:", str(e))


In [ ]:
df_carros

In [ ]:
#df_carros['km_cidade'] = df_carros['city_mpg'] * 1.60934
#df_carros['km_estrada'] = df_carros['highway_mpg'] * 1.60934
#df_carros['litro_cidade'] = df_carros['city_mpg'] * 3.78541
#df_carros['litro_estrada'] = df_carros['highway_mpg'] * 3.78541

In [ ]:
## Convertendo milha para Km - (1 milha equivale a aproximadamente 1,60934 km)
km_cidade = df_carros['city_mpg'] * 1.60934
km_estrada = df_carros['highway_mpg'] * 1.60934

## Convertendo galao para litro - (1 galão equivale a aproximadamente 3,78541 litros)
litro_cidade = df_carros['city_mpg'] * 3.78541
litro_estrada = df_carros['highway_mpg'] * 3.78541

km_cidade =  km_cidade.mean()
km_estrada =  km_estrada.mean()
litro_cidade = litro_cidade.mean()
litro_estrada = litro_estrada.mean()

In [ ]:
res = litro_cidade / km_cidade
res

In [ ]:
# Calculando o consumo em litros por quilômetro (L/km)
df['consumo_litros_por_km_cidade'] = litro_cidade / km_cidade
df['consumo_litros_por_km_estrada'] = litro_estrada / km_estrada

In [ ]:
# Calculando o consumo em km/L
df['consumo_km_por_litro_cidade'] = km_cidade / litro_cidade
df['consumo_km_por_litro_estrada'] = km_estrada / litro_estrada

# Inicio da Utilização de modelos de ML

## Regressão Linear

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pandas as pd
from minio import Minio
import io
from io import StringIO, BytesIO
import psycopg2
from datetime import datetime
data_hora_atual = datetime.now()
REGRESSAO_LINEAR = 'regressao-linear'
ML_RESULTS = 'resultados-ml'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

db_config = {
'host': 'localhost',
'database': 'postgres',
'user': 'postgres',
'password': 'postgres',
}

In [ ]:
# Fazendo conexão com o banco para selecionar a view "vw_valores_combustivel_por_viagem" com os dados que serão utilizados para a regressão linear.
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

consulta_veiculos = """
select 
	data_rota,	
	nome_usuario,
	tipo_combustivel,	
	dist_km,
	cons_por_km as consumo_litros_viagem	
from vw_valores_combustivel_por_viagem 
"""

cursor.execute(consulta_veiculos)
resultados_consumo_veiculos = cursor.fetchall()
conn.close()
df_consumo_veiculos = pd.DataFrame(resultados_consumo_veiculos, columns=[desc[0] for desc in cursor.description])

In [ ]:
df_consumo_veiculos

In [ ]:
# Definindo variáveis independentes (X) e dependente (Y)
X = df_consumo_veiculos[['dist_km','nome_usuario']]
Y = df_consumo_veiculos['consumo_litros_viagem']
X = pd.get_dummies(X, columns=['nome_usuario'], drop_first=True)

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

# Inicializando o modelo de regressão linear
model = LinearRegression()

# Treinando o modelo
model.fit(X_train, Y_train)

# Fazendo previsões no conjunto de teste
Y_pred = model.predict(X_test)

# Avaliando o desempenho do modelo
print('Erro Médio Absoluto:', metrics.mean_absolute_error(Y_test, Y_pred))
print('Erro Quadrático Médio:', metrics.mean_squared_error(Y_test, Y_pred))


coeficientes = model.coef_
coeficientes_df = pd.DataFrame({'Variável': X.columns, 'Coeficiente': coeficientes})
coeficientes_df['Coeficiente_Abs'] = coeficientes_df['Coeficiente'].abs().apply(lambda x: format(x, '.6f'))
coeficientes_df = coeficientes_df.sort_values(by='Coeficiente_Abs', ascending=False)

#valor_formatado = format(5.551115e-17, '.10f')
#valor_formatado
#valor_arredondado = round(5.551115e-17, 10)
#valor_arredondado

# Converta o DataFrame enriquecido de volta para CSV
csv_to_regressao_linear = coeficientes_df.to_csv(index=False, sep=';')
csv_to_regressao_linear_bytes = csv_to_regressao_linear.encode('utf-8')

# Crie um buffer de bytes
csv_to_regressao_linear_buffer = BytesIO(csv_to_regressao_linear_bytes)
nome_arquivo = f'regressao_linear_{data_hora_atual}.csv'

minioclient.put_object(
        REGRESSAO_LINEAR,
        nome_arquivo,  # Use o mesmo nome de arquivo
        data=csv_to_regressao_linear_buffer,
        length=len(csv_to_regressao_linear_bytes),
        content_type='application/csv')

## Enviando a regressão Linear para o Banco 
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

truncate = """ truncate table regressao_linear_temp; """
cursor.execute(truncate)

copy_sql = """
    COPY regressao_linear_temp (variavel, coeficiente, coeficiente_abs)
    FROM stdin WITH CSV HEADER DELIMITER as ';'"""


arquivos_regressao = [arquivo for arquivo in minioclient.list_objects(REGRESSAO_LINEAR) if arquivo.object_name.endswith(".csv")]
# Itera sobre cada arquivo CSV encontrado no Minio
for arquivo_regressao in arquivos_regressao:
    # Obtém o objeto do arquivo CSV do Minio  
    obj_regr = minioclient.get_object(REGRESSAO_LINEAR, arquivo_regressao.object_name)
    csv_decod = obj_regr.data.decode('utf-8')  # Convertendo bytes para string
    arquivo_csv = StringIO(csv_decod)
    df = pd.read_csv(arquivo_csv, sep=';')
    csv_bytes = df.to_csv(index=False,sep=';').encode('utf-8')
    csv_buffer = BytesIO(csv_bytes)
    nome_arquivo = arquivo_regressao.object_name

    with io.StringIO(csv_decod) as file:
        cursor.copy_expert(sql=copy_sql, file=file)
    conn.commit()

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
insert = """
insert into regressao_linear (variavel, coeficiente, coeficiente_abs, data_execucao )
select 	replace (variavel,'nome_usuario_','') as variavel, 
		Coeficiente, 
		Coeficiente_Abs, 
		now() as data_execucao 
from regressao_linear_temp
    """
cursor.execute(insert)
conn.commit()
conn.close()

minioclient.put_object(
        ML_RESULTS,
        arquivo_regressao.object_name,  # Use o mesmo nome de arquivo
        data=csv_buffer,
        length=len(csv_bytes),
        content_type='application/csv')

minioclient.remove_object(REGRESSAO_LINEAR, nome_arquivo)

#### Com essa Regressão feita temos acesso a um indicativo do veículo que mais consome combustível 
#### Pode-se considerar que as informações acima do dist_km são influenciadas pelo nome_usuario e abaixo do dist_km são influenciadas pela distancia

#### Entre os tipos de veículos, o RAM_1500_5TH_GEN tem o maior impacto absoluto no consumo de combustível, com um coeficiente absoluto de 0.507708.

#### Quando consideramos a distância (dist_km), o veículo do tipo HYUNDAI_SANTA_FE_HYBRID_2022 tem o maior impacto absoluto no consumo de combustível entre os veículos listados, com um coeficiente absoluto de 0.092526.

#### Essas interpretações são feitas com base nos coeficientes do modelo de regressão linear que você treinou. Vale ressaltar que o impacto é relativo às outras categorias ou variáveis incluídas no modelo. Portanto, para a variável específica "nome_usuario_HYUNDAI_SANTA_FE_HYBRID_2022", o coeficiente absoluto indica a magnitude do impacto dessa categoria em relação à categoria de referência (ou base) para essa variável.

## Agrupamento (Clustering)

### Versão Funcional

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
from minio import Minio
import io
from io import StringIO, BytesIO
import psycopg2
from datetime import datetime
data_hora_atual = datetime.now()
from time import sleep
from decimal import Decimal
CLUSTERING = 'clustering'
ML_RESULTS = 'resultados-ml'


minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)


db_config = {
'host': 'localhost',
'database': 'postgres',
'user': 'postgres',
'password': 'postgres',
}

In [ ]:
sql_query = """
select	
	nome_usuario,
	case when tipo_combustivel = 'gas' then 1 end as tipo_combustivel, 	
	dist_km,
	media_consumo_km_l, 
	case when custo_trajeto = null then avg(custo_trajeto::numeric) else custo_trajeto end as custo_trajeto  	
from vw_valores_combustivel_por_viagem
where tipo_combustivel = 'gas'
group by
	nome_usuario,
	tipo_combustivel, 	
	dist_km,
	media_consumo_km_l,
	custo_trajeto 
"""


conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
cursor.execute(sql_query)
resultados_agrup_veiculos = cursor.fetchall()
conn.close()
df_agrup_veiculos = pd.DataFrame(resultados_agrup_veiculos, columns=[desc[0] for desc in cursor.description])

In [ ]:
df_agrup_veiculos

In [ ]:
# Selecionando colunas relevantes
X = df_agrup_veiculos[['dist_km', 'media_consumo_km_l']]


# Normalizando os dados
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)


# Especificando o número de clusters
num_clusters = 3  


# Aplicando o K-means
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
df_agrup_veiculos['cluster_consumo'] = kmeans.fit_predict(X_normalized)


# Criando um novo DataFrame com os resultados do agrupamento
df_veiculos_clusterizados = df_agrup_veiculos[['nome_usuario', 'dist_km', 'media_consumo_km_l', 'cluster_consumo']].copy()


# Adicionando informações adicionais conforme necessário
# df_resultado['informacao_adicional'] = ...

# Salvando o DataFrame resultante em um arquivo CSV ou inserindo no banco de dados
df_veiculos_clusterizados.to_csv('resultado_agrupamento.csv', index=False)


##--------------------------------------------------------------------------------------------------##
##--------------------------------------------------------------------------------------------------##

## Calculando a média do consumo de combustível para cada cluster
#media_consumo_por_cluster = df_agrup_veiculos.groupby('cluster_consumo')['media_consumo_km_l'].mean()
## Exibindo os resultados
#for cluster, media_consumo in media_consumo_por_cluster.items():
#    print(f"\nCluster {cluster} - Média de Consumo: {media_consumo}")
#    veiculos_cluster = df_agrup_veiculos[df_agrup_veiculos['cluster_consumo'] == cluster]
#    print(veiculos_cluster[['nome_usuario', 'media_consumo_km_l']])

##--------------------------------------------------------------------------------------------------##
##--------------------------------------------------------------------------------------------------##


## Converta o DataFrame enriquecido de volta para CSV
csv_veiculos_clusterizados = df_veiculos_clusterizados.to_csv(index=False, sep=';')
csv_veiculos_clusterizados_bytes = csv_veiculos_clusterizados.encode('utf-8')

## Crie um buffer de bytes
csv_veiculos_clusterizados_buffer = BytesIO(csv_veiculos_clusterizados_bytes)
nome_arquivo = f'clustering_{data_hora_atual}.csv'

minioclient.put_object(
        CLUSTERING,
        nome_arquivo,  # Use o mesmo nome de arquivo
        data=csv_veiculos_clusterizados_buffer,
        length=len(csv_veiculos_clusterizados_bytes),
        content_type='application/csv')

In [ ]:
## Enviando a regressão Linear para o Banco 
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

truncate = """ truncate table veiculos_clusterizados_temp; """
cursor.execute(truncate)

copy_sql = """
    COPY veiculos_clusterizados_temp (nome_usuario, dist_km, media_consumo_km_l, cluster_consumo)
    FROM stdin WITH CSV HEADER DELIMITER as ';'"""

arquivos_clustering = [arquivo for arquivo in minioclient.list_objects(CLUSTERING) if arquivo.object_name.endswith(".csv")]
## Itera sobre cada arquivo CSV encontrado no Minio
for arquivo_clustering in arquivos_clustering:
    # Obtém o objeto do arquivo CSV do Minio  
    obj_regr = minioclient.get_object(CLUSTERING, arquivo_clustering.object_name)
    csv_decod = obj_regr.data.decode('utf-8')  # Convertendo bytes para string
    arquivo_csv = StringIO(csv_decod)
    df = pd.read_csv(arquivo_csv, sep=';')
    csv_bytes = df.to_csv(index=False,sep=';').encode('utf-8')
    csv_buffer = BytesIO(csv_bytes)
    nome_arquivo = arquivo_clustering.object_name

    with io.StringIO(csv_decod) as file:
        cursor.copy_expert(sql=copy_sql, file=file)
conn.commit()
conn.close()

In [ ]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
insert = """
        insert into veiculos_clusterizados (nome_usuario, dist_km, media_consumo_km_l, cluster_consumo, data_execucao )
        select 	
        	nome_usuario,
        	dist_km,
        	media_consumo_km_l,
        	cluster_consumo,
        	now() as data_execucao 
        from veiculos_clusterizados_temp
    """
cursor.execute(truncate)
cursor.execute(insert)
conn.commit()
conn.close()

minioclient.put_object(
        ML_RESULTS,
        arquivo_clustering.object_name,  # Use o mesmo nome de arquivo
        data=csv_buffer,
        length=len(csv_bytes),
        content_type='application/csv')


minioclient.remove_object(CLUSTERING, arquivo_clustering.object_name)

In [ ]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
pior_desempenho = """
select 
	nome_usuario,
	round(media_consumo_km_l::numeric,2) as media_consumo_km_l,
	round(sum(dist_km::numeric),2) as dist_km, 
	round(sum((dist_km::numeric / media_consumo_km_l::numeric) * (select AVG(preco_medio_revenda::NUMERIC) from preco_combustivel_semanal where estado = 'RIO DE JANEIRO' and produto like '%GAS%')),2) as custo_viagem 
from veiculos_clusterizados
where cluster_consumo between '0' and '1'
group by nome_usuario,
	media_consumo_km_l
order by media_consumo_km_l::numeric asc
limit 5
"""
cursor.execute(pior_desempenho)
resultados_pior_desempenho = cursor.fetchall()
conn.close()
df_pior_desempenho = pd.DataFrame(resultados_pior_desempenho, columns=[desc[0] for desc in cursor.description])
df_pior_desempenho = df_pior_desempenho.sort_values(by='custo_viagem')

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
melhor_desempenho = """
select 
	nome_usuario,
	round(media_consumo_km_l::numeric, 2) as media_consumo_km_l,
	round(sum(dist_km::numeric),2) as dist_km, 
	round(sum((dist_km::numeric / media_consumo_km_l::numeric) * (select AVG(preco_medio_revenda::NUMERIC) from preco_combustivel_semanal where estado = 'RIO DE JANEIRO' and produto like '%GAS%')),2) as custo_viagem  
from veiculos_clusterizados
where cluster_consumo = '2'
group by nome_usuario,
	media_consumo_km_l	
order by media_consumo_km_l::numeric desc
limit 5
"""
cursor.execute(melhor_desempenho)
resultados_melhor_desempenho = cursor.fetchall()
conn.close()
df_melhor_desempenho = pd.DataFrame(resultados_melhor_desempenho, columns=[desc[0] for desc in cursor.description])
df_melhor_desempenho = df_melhor_desempenho.sort_values(by='custo_viagem')

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
preco_medio_gasolina = """
select 
	avg(preco_medio_revenda::numeric) as media
from preco_combustivel_semanal
where
	 produto like '%GASOLINA%' and 
     estado = 'RIO DE JANEIRO'
"""
cursor.execute(preco_medio_gasolina)
resultados_preco_medio_gasolina = cursor.fetchall()
conn.close()
df_preco_medio_gasolina = pd.DataFrame(resultados_preco_medio_gasolina, columns=[desc[0] for desc in cursor.description])
valor_medio = Decimal(df_preco_medio_gasolina['media'].iloc[0])


In [ ]:
df_pior_desempenho

In [ ]:
df_melhor_desempenho

In [ ]:
from decimal import Decimal
valor_medio = Decimal(df_preco_medio_gasolina['media'].iloc[0])
valor_medio

In [ ]:
df_simulado = pd.DataFrame({
    'nome_usuario_original': df_pior_desempenho['nome_usuario'],
    'media_consumo_km_original': df_pior_desempenho['media_consumo_km_l'],
    'nome_usuario_simulado': df_melhor_desempenho['nome_usuario'],
    'media_consumo_km_l_simulado': df_melhor_desempenho['media_consumo_km_l'],
    'dist_km_original': df_pior_desempenho['dist_km'],
    'custo_viagem_original': df_pior_desempenho['custo_viagem'],    
    'custo_viagem_simulado': df_pior_desempenho['dist_km'] / df_melhor_desempenho['media_consumo_km_l'] * valor_medio
    })
df_simulado

In [ ]:
## Converta o DataFrame enriquecido de volta para CSV
csv_simulado = df_simulado.to_csv(index=False, sep=';')
csv_simulado_bytes = csv_simulado.encode('utf-8')

## Crie um buffer de bytes
csv_simulado_buffer = BytesIO(csv_simulado_bytes)
nome_arquivo = f'simulacao_rota_veiculos_{data_hora_atual}.csv'

minioclient.put_object(
        CLUSTERING,
        nome_arquivo,  # Use o mesmo nome de arquivo
        data=csv_simulado_buffer,
        length=len(csv_simulado_bytes),
        content_type='application/csv')

## Enviando a simulação para o Banco 
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

truncate = """ truncate table cluster_simulacao; """
cursor.execute(truncate)

copy_sql = """
    COPY cluster_simulacao (nome_usuario_original,media_consumo_km_original,nome_usuario_simulado,media_consumo_km_l_simulado,dist_km_original,custo_viagem_original,custo_viagem_simulado)
    FROM stdin WITH CSV HEADER DELIMITER as ';'
    """

arquivos_clustering = [arquivo for arquivo in minioclient.list_objects(CLUSTERING) if arquivo.object_name.endswith(".csv")]
## Itera sobre cada arquivo CSV encontrado no Minio
for arquivo_clustering in arquivos_clustering:
    # Obtém o objeto do arquivo CSV do Minio  
    obj_regr = minioclient.get_object(CLUSTERING, arquivo_clustering.object_name)
    csv_decod = obj_regr.data.decode('utf-8')  # Convertendo bytes para string
    arquivo_csv = StringIO(csv_decod)
    df = pd.read_csv(arquivo_csv, sep=';')
    csv_bytes = df.to_csv(index=False,sep=';').encode('utf-8')
    csv_buffer = BytesIO(csv_bytes)
    nome_arquivo = arquivo_clustering.object_name

    with io.StringIO(csv_decod) as file:
        cursor.copy_expert(sql=copy_sql, file=file)
conn.commit()
conn.close()

In [ ]:
arquivos_clustering = [arquivo for arquivo in minioclient.list_objects(CLUSTERING) if arquivo.object_name.endswith(".csv")]
## Itera sobre cada arquivo CSV encontrado no Minio
for arquivo_clustering in arquivos_clustering:
    # Obtém o objeto do arquivo CSV do Minio      
    nome_arquivo = arquivo_clustering.object_name    
    minioclient.remove_object(CLUSTERING, nome_arquivo)



In [ ]:
minioclient.put_object(
        ML_RESULTS,
        arquivo_clustering.object_name,  # Use o mesmo nome de arquivo
        data=csv_buffer,
        length=len(csv_bytes),
        content_type='application/csv')

minioclient.remove_object(CLUSTERING, nome_arquivo)

# Nova versão da distancia de Manhattan

In [1]:
import pandas as pd
from minio import Minio
import io
from io import StringIO, BytesIO
import psycopg2
from datetime import datetime
data_hora_atual = datetime.now()
from time import sleep
from decimal import Decimal
from scipy.spatial.distance import  minkowski


minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)


db_config = {
'host': 'localhost',
'database': 'postgres',
'user': 'postgres',
'password': 'postgres',
}

# Cálculo da distância Euclidiana
def calc_dist_euclidiana(row,p=2):
    coord1 = (float(row['first_long']), float(row['first_lat']))
    coord2 = (float(row['last_long']), float(row['last_lat']))
    return minkowski(coord1, coord2, p)

# Cálculo da distância de Manhattan
def calc_dist_manhattan(row, p=1):
    coord1 = (float(row['first_long']), float(row['first_lat']))
    coord2 = (float(row['last_long']), float(row['last_lat']))
    return minkowski(coord1, coord2, p)

# Cálculo da distância de Minkowski
def calc_dist_minkowski(row, p=3):
    coord1 = (float(row['first_long']), float(row['first_lat']))
    coord2 = (float(row['last_long']), float(row['last_lat']))
    return minkowski(coord1, coord2, p)

In [2]:
sql_query = """
with euclidean_manhattan_minkowski as (
	SELECT 
		distinct 	
		id_rota,
		nome_usuario,
		data_rota, 
		cidade,
		estado,
		pais,
		first_value(hora_rota) OVER (PARTITION BY id_rota ORDER BY id_rota)::time without time zone AS inicio_rota,
		last_value(hora_rota) OVER (PARTITION BY id_rota ORDER BY id_rota)::time without time zone AS fim_rota,
		first_value(longitude) OVER (PARTITION BY id_rota ORDER BY id_rota) AS first_long,
		last_value(longitude) OVER (PARTITION BY id_rota ORDER BY id_rota) AS last_long,
		first_value(latitude) OVER (PARTITION BY id_rota ORDER BY id_rota) AS first_lat,
		last_value(latitude) OVER (PARTITION BY id_rota ORDER BY id_rota) AS last_lat
FROM tb_gpx_full)

SELECT
	id_rota,
    nome_usuario,
    data_rota,
    inicio_rota,
    fim_rota,
	case 
		when fim_rota > inicio_rota then fim_rota - inicio_rota
        when inicio_rota > fim_rota then inicio_rota - fim_rota 
    end as tempo_viagem,
    cidade,
    estado,
    pais,
    first_long,
	last_long,
	first_lat,
	last_lat
FROM euclidean_manhattan_minkowski
"""

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
cursor.execute(sql_query)
pre_manhattan = cursor.fetchall()
conn.close()
df_pre_manhattan = pd.DataFrame(pre_manhattan, columns=[desc[0] for desc in cursor.description])

In [ ]:
df_pre_manhattan.head()
#print(df_pre_manhattan.dtypes)

In [ ]:
df_pre_manhattan[['first_long', 'first_lat', 'last_long', 'last_lat']] = df_pre_manhattan[['first_long', 'first_lat', 'last_long', 'last_lat']].astype(float).round(10)

df_pre_manhattan['dist_euclidiana'] = df_pre_manhattan.apply(calc_dist_euclidiana, axis=1)
df_pre_manhattan['dist_manhattan'] = df_pre_manhattan.apply(calc_dist_manhattan, axis=1)
df_pre_manhattan['dist_minkowski'] = df_pre_manhattan.apply(lambda row: calc_dist_minkowski(row, p=3), axis=1)  

df_pre_manhattan['dist_euclidiana'] *= 1000
df_pre_manhattan['dist_manhattan'] *= 1000
df_pre_manhattan['dist_minkowski'] *= 1000

df_euclidean_manhattan_minkowski = df_pre_manhattan
df_euclidean_manhattan_minkowski.head()



df_filtrado = df_euclidean_manhattan_minkowski[
    (df_euclidean_manhattan_minkowski['data_rota'].notnull()) &
    (df_euclidean_manhattan_minkowski['inicio_rota'].notnull()) &
    (df_euclidean_manhattan_minkowski['id_rota'] == '11189161') &
    (df_euclidean_manhattan_minkowski['nome_usuario'] == 'FORD_MUSTANG')
]
df_filtrado

In [ ]:
df_euclidean_manhattan_minkowski

In [ ]:
df_pre_manhattan.reset_index(drop=True, inplace=True)


In [ ]:
df = pd.read_csv('11190523__routes__HYUNDAI_IONIQ_5_2022.csv', sep=';')
df['lat1'] = df['latitude'].shift()
df['lon1'] = df['longitude'].shift()
df = df.rename(columns={'latitude': 'lat2', 'longitude': 'lon2'})
df

In [ ]:
df = df.dropna()
df

#### Clusters Identificados:

#### Cada linha do DataFrame agora está associada a um cluster (grupo identificado pelo K-means). Esses clusters representam grupos de viagens com padrões semelhantes de consumo de combustível e distância percorrida.

#### Comparação entre Veículos:
#### Ao analisar o DataFrame agrupado, você pode identificar quais veículos ou usuários estão no mesmo cluster. Isso indica que esses veículos têm comportamentos semelhantes em termos de eficiência de combustível e distância percorrida.

#### Eficiência de Combustível e Distância:
#### Dentro de cada cluster, observe a variação nos valores de dist_km e media_consumo_km_l. Isso ajuda a entender como diferentes veículos se comportam em termos de eficiência de combustível para distâncias específicas.

#### Detecção de Padrões Anômalos:
#### Viagens que não seguem o padrão esperado podem ser identificadas. Por exemplo, se a maioria dos veículos em um cluster possui alta eficiência de combustível para uma determinada distância, viagens que se desviam desse padrão podem ser investigadas.

#### Otimização de Rotas e Consumo:
#### Se você tiver informações adicionais sobre as rotas (por exemplo, cidades, estradas, condições de tráfego), pode-se explorar se o clusterização reflete padrões relacionados a esses fatores, o que poderia ajudar na otimização de rotas e consumo.

#### Exemplos Claros:
#### Cluster 1: Eficientes em Cidade:
#### Veículos neste cluster podem ter alta eficiência de combustível em distâncias curtas, indicando eficiência em ambientes urbanos.

#### Cluster 2: Viagens de Longa Distância:
#### Este cluster pode conter veículos que são eficientes em viagens de longa distância, indicando bom desempenho em rodovias.

#### Cluster 3: Comportamento Variado:
#### Viagens neste cluster podem ter uma combinação de eficiência de combustível e distâncias, representando comportamentos variados.

#### Justificativa para Utilização:
#### Personalização de Estratégias:
#### Compreender esses padrões permite personalizar estratégias de otimização, como fornecer recomendações específicas para motoristas ou ajustar políticas de manutenção.

#### Identificação de Tendências de Consumo:
#### Ajuda a identificar tendências gerais no consumo de combustível, proporcionando insights valiosos para tomada de decisões informadas.

#### Economia de Recursos:
#### Pode levar a economias significativas otimizando rotas, manutenção preventiva e promoção de práticas eficientes.
#### Ao interpretar os resultados, é essencial considerar o contexto específico do seu conjunto de dados e os objetivos do seu negócio. A análise pode ser refinada com mais detalhes conforme você explora mais características e contextos específicos.

### Testes Clustering

In [ ]:
X = df_agrup_veiculos[['dist_km', 'tipo_combustivel', 'custo_trajeto']]

In [ ]:
scaler = StandardScaler()
type(scaler)

In [ ]:
X_normalized = scaler.fit_transform(X)
type(X_normalized)

In [ ]:

num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters)
kmeans

In [ ]:
kmeans.fit(X_normalized)
kmeans

In [ ]:
df_agrup_veiculos['nome_usuario'] = kmeans.labels_
df_agrup_veiculos

In [ ]:
X = df_agrup_veiculos[['nome_usuario','dist_km', 'tipo_combustivel', 'custo_trajeto']]
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(X_normalized)
df_agrup_veiculos['inicio_viagem'] = kmeans.labels_
df_agrup_veiculos


In [ ]:
df_agrup_veiculos.groupby('inicio_viagem').describe()

In [ ]:
sns.scatterplot(data=df_agrup_veiculos, x='nome_usuario', y='custo_trajeto', hue='inicio_viagem', palette='viridis')
plt.title('Consumo de Combustível por Cluster')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Selecionando as features relevantes
X = df_agrup_veiculos[['dist_km', 'media_consumo_km_l']]

# Normalizando os dados
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Especificando o número de clusters (você pode ajustar conforme necessário)
num_clusters = 3

# Aplicando o K-means
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
df_agrup_veiculos['cluster_consumo'] = kmeans.fit_predict(X_normalized)

# Exibindo o resultado
print(df_agrup_veiculos[['dist_km', 'media_consumo_km_l', 'cluster_consumo']])


In [ ]:
# Calculando a média do consumo de combustível para cada cluster
media_consumo_por_cluster = df_agrup_veiculos.groupby('cluster_consumo')['media_consumo_km_l'].mean()

# Identificando os veículos que estão acima da média em cada cluster
veiculos_mais_consumo = {}
for cluster, media_consumo in media_consumo_por_cluster.items():
    veiculos_cluster = df_agrup_veiculos[df_agrup_veiculos['cluster_consumo'] == cluster]
    veiculos_acima_media = veiculos_cluster[veiculos_cluster['media_consumo_km_l'] > media_consumo]
    veiculos_mais_consumo[cluster] = veiculos_acima_media[['nome_usuario', 'media_consumo_km_l']]

# Exibindo os resultados
for cluster, veiculos in veiculos_mais_consumo.items():
    print(f"\nCluster {cluster} - Veículos que mais consumiram:")
    print(veiculos)


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Selecione apenas as colunas necessárias para a análise
df_agrup_veiculos = df_agrup_veiculos[['dist_km', 'media_consumo_km_l', 'nome_usuario']].copy()

# Remova duplicatas se existirem
df_agrup_veiculos.drop_duplicates(subset=['dist_km', 'media_consumo_km_l', 'nome_usuario'], inplace=True)

# Normalizar os dados
X = df_agrup_veiculos[['dist_km', 'media_consumo_km_l']]
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Especificar o número de clusters
num_clusters = 3

# Aplicar K-means
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
df_agrup_veiculos['cluster_consumo'] = kmeans.fit_predict(X_normalized)

# Calcular a média do consumo de combustível para cada cluster
media_consumo_por_cluster = df_agrup_veiculos.groupby('cluster_consumo')['media_consumo_km_l'].mean()

# Identificar os veículos que estão acima da média em cada cluster
veiculos_mais_consumo = {}
for cluster, media_consumo in media_consumo_por_cluster.items():
    veiculos_cluster = df_agrup_veiculos[df_agrup_veiculos['cluster_consumo'] == cluster]
    veiculos_acima_media = veiculos_cluster[veiculos_cluster['media_consumo_km_l'] > media_consumo]
    veiculos_mais_consumo[cluster] = veiculos_acima_media[['nome_usuario', 'media_consumo_km_l']]

# Exibir os resultados
for cluster, veiculos in veiculos_mais_consumo.items():
    print(f"\nCluster {cluster} - Veículos que mais consumiram:")
    print(veiculos)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualizando os clusters
sns.scatterplot(data=df_agrup_veiculos, x='dist_km', y='media_consumo_km_l', hue='cluster_consumo', palette='viridis')
plt.title('Agrupamento de Viagens Semelhantes')
plt.xlabel('Distância (km)')
plt.ylabel('Consumo Médio (km/l)')
plt.show()


## Aprendizado não supervisionado com K-means e Voronoi

# Códigos não mais usados

In [ ]:
## Aqui os arquivos saem da pasta download para a pasta DATALAKE_STAGE

# files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
# for nome_arquivos in files_to_move:
#     origin_path = os.path.join(DOWNLOADS, nome_arquivos)
#     destiny_path = os.path.join(DATALAKE_STAGE, nome_arquivos)
#     os.rename(origin_path, destiny_path)
#     print(f"Arquivo {nome_arquivos} movido para {destiny_path}")

In [ ]:
## Aqui os arquivos saem da pasta DATALAKE_STAGE para o bucket MinIO

# for item in os.listdir(DATALAKE_STAGE):
#     caminho_pre_proc = os.path.join(DATALAKE_STAGE, item)
#     print(caminho_pre_proc)
# 
#     if os.path.isfile(caminho_pre_proc):
#         try:
#             minioclient.fput_object(DATA_LAKE, item, caminho_pre_proc)
#             print(f"Arquivo {item} enviado com sucesso para o bucket.")
#         except S3Error as err:
#             print(f"Erro ao enviar o arquivo {item}: {err}")

In [ ]:
# ## DADOS PARA INSERIR NA TABELA DE CONTROLE USANDO O BEAUTIFULSOAP ## #
# navegador = requests.get('https://www.openstreetmap.org/user/sunnypilot/traces/9255974')
# site = BeautifulSoup(navegador.text, 'html.parser')
# data_trace = site.find('div', attrs={'class': 'content-body'})
# table_trace = data_trace.find('table')
# 
# filename_row = table_trace.find('th', string='Filename:').parent
# owner_row = table_trace.find('th', string='Owner:').parent
# uploaded_row = table_trace.find('th', string='Uploaded:').parent
# 
# filename = filename_row.find('td').text.strip().replace('(download)','').strip()
# owner = owner_row.find('td').text.strip()
# uploaded = uploaded_row.find('td').text.strip()

In [ ]:
# ## ESTE TRECHO TEM A FINALIDADE DE BAIXAR OS ARQUIVOS ## #
#list_rotas_finalizadas
#list_rotas_pendentes

# //*[@id="content"]/div[2]/div/table/tbody/tr[6]/td - finalizado
# //*[@id="content"]/div[2]/div/span = PENDENTE
# //*[@id="content"]/div[2]/div/table/tbody/tr[4]/td - pendente

# ## Trecho descontinuado -- Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
# navegador = webdriver.Chrome(service=servico)
# for list_route in lista_rotas:
#     sleep(3)
#     url = list_route[0]
#     navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
#     navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
# navegador.close()

In [ ]:
# pip install pyspark
from pyspark.sql import SparkSession
# Criando as variáveis que serão utilizadas no Spark
appname = 'tcc-project'
master = 'local'
# Criando a sessão Spark
spark = SparkSession.builder\
    .appName(appname)\
    .master(master)\
    .getOrCreate()

In [ ]:
# Criando a sessão Spark
appname = 'tcc-project'
master = 'local'
spark = SparkSession.builder\
    .appName(appname)\
    .master(master)\
    .getOrCreate()

In [ ]:
spark

In [ ]:
# pip install pyspark
from pyspark.sql import SparkSession
# Criando as variáveis que serão utilizadas no Spark
appname = 'tcc-project'
master = 'local'
# Criando a sessão Spark
spark = SparkSession.builder\
    .appName(appname)\
    .master(master)\
    .getOrCreate()

spark
# Coalesce o DataFrame para um único arquivo
df_single_file = df.coalesce(1)
# Salve o DataFrame coalesced como um único arquivo CSV
df_single_file.write.csv(csv_caminho_arquivo, header=True, mode="overwrite")
# Dessa forma o spark gera um csv particionado
df.write.csv('pyspark_dataframe.csv', header=True, mode="overwrite")